# Foundry name: MUNJAL

In [7]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

#loading the prepared sand data
dfp = pd.read_excel('Preparedsand_01-Jan-2019_TO_15-Mar-2020_Disa 231Y.xlsx',skiprows=[0,1,2,3,4])
dfp.drop(dfp.columns[[3, 14, 27]], axis = 1, inplace = True)#removing heat number, specimen weight and relative humidity
dfp = dfp.dropna(axis=1,how='all') #removing the columns with full null values
dfp.fillna(dfp.median(), inplace=True)  #replacing rest of the null values with median values

dfb = pd.read_excel('Bentonite_01-Jan-2019_TO_15-Mar-2020_Disa 231Y.xlsx',skiprows=[0,1,2,3,4])
dfb.drop(dfb.columns[[1, 2, 3, 4]], axis = 1, inplace = True)#removing batch no, supplier, quantity, and grade columns
dfb.columns = dfb.columns.str.replace(' ','_').str.replace('(','').str.replace(')','').str.replace('%','_raw').str.replace('kg/cm2','_raw').str.replace('gm/cm2','_raw')
dfb = dfb.dropna(axis=1,how='all') #removing the columns with full null values
dff_b = pd.merge(dfp,dfb,on='Date',how='left')
dff_b.fillna(method='ffill',inplace=True)
dff_b.fillna(method='bfill',inplace=True)
cols = list(dff_b)
cols[0],cols[1],cols[2],cols[3],cols[4],cols[5],cols[6],cols[7],cols[8],cols[9],cols[10],cols[11],cols[12],cols[13],cols[14],cols[15],cols[16],cols[17],cols[18],cols[19],cols[20],cols[21] = cols[0],cols[1],cols[2],cols[3],cols[4],cols[5],cols[6],cols[7],cols[9],cols[13],cols[8],cols[10],cols[11],cols[12],cols[14],cols[15],cols[16],cols[17],cols[18],cols[19],cols[20],cols[21]
dff_b = dff_b.loc[:,cols]
df1 = dff_b.copy()
db = dff_b.copy()
df1.drop(df1.columns[[0,1,2,3]],axis=1,inplace=True)
dfs = pd.read_excel('Freshsilicasand_01-Jan-2019_TO_15-Mar-2020_Disa 231Y.xlsx',skiprows=[0,1,2,3,4]) #loading FSS raw material data
dfs.drop(dfs.columns[[1,2,3,5,8]], axis = 1, inplace = True)
dfs.columns = dfs.columns.str.replace(' ','_').str.replace('(','').str.replace(')','').str.replace('%','_raw').str.replace('kg/cm2','_raw').str.replace('gm/cm2','_raw').str.replace('_no','_raw')
dfs = dfs.dropna(axis=1,how='all') #removing the columns with full null values
dff_fs = pd.merge(dfp,dfs,on='Date',how='left')
dff_fs.fillna(method='ffill',inplace=True)
dff_fs.fillna(method='bfill',inplace=True)
cols = list(dff_fs)
cols[0],cols[1],cols[2],cols[3],cols[4],cols[5],cols[6],cols[7],cols[8],cols[9],cols[10],cols[11],cols[12],cols[13],cols[14],cols[15],cols[16],cols[17] = cols[0],cols[1],cols[2],cols[3],cols[14],cols[4],cols[5],cols[6],cols[7],cols[8],cols[9],cols[10],cols[11],cols[12],cols[13],cols[15],cols[16],cols[17]
dff_fs = dff_fs.loc[:,cols]
dfs = dff_fs.copy()

df3 = dff_fs.copy()
df3.drop(df3.columns[[0,1,2,3]],axis=1,inplace=True)
cols = list(dfp)
cols[0],cols[1],cols[2],cols[3],cols[4],cols[5],cols[6],cols[7],cols[8],cols[9],cols[10],cols[11],cols[12],cols[13],cols[14],cols[15],cols[16] = cols[0],cols[1],cols[2],cols[3],cols[8],cols[11],cols[12],cols[4],cols[5],cols[6],cols[7],cols[9],cols[10],cols[13],cols[14],cols[15],cols[16]
dfp = dfp.loc[:,cols]
df2 = dfp.copy()
dpr = dfp.copy()
df2.drop(df2.columns[[0,1,2,3]],axis=1,inplace=True)
#df1 has prepared sand and bentonite data
#df2 has only the prepared sand data
#df3 has prepared sand and FSS data
d1 = df1.copy()
d1.drop(d1.columns[[7, 11,12,13,14,15,16,17]], axis = 1, inplace = True)
rf = RandomForestRegressor(n_estimators=100,random_state=0)
boost_rf = AdaBoostRegressor(base_estimator = rf, n_estimators=50,random_state=0)
df1.columns = df1.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
df2.columns = df2.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
df3.columns = df3.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
d1.columns = d1.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in df1:
    X=df1.drop(i,axis=1) #removing the target variable
    y = df1[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in df2:
    X=df2.drop(i,axis=1)
    y = df2[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
#similar to df1 and df2 , all the same steps are carried out for df3
for i in df3:
    X=df3.drop(i,axis=1)
    y = df3[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'All')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)
#Groupwise:

dff_b.columns = dff_b.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
dff_fs.columns = dff_fs.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
dfp.columns = dfp.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')

group_low_b= dff_b.loc[dff_b['component_id'].isin(['Brake Disc _Cav 7:&8: (Model K)',
          'Brake Disc, (Cav 7,8,9,10)Alto',
          'Crank Case',
        'Cylinder Block Cast_new pattern',
         'Cylinder Block _ BMG',
          'Cylinder Block _ CMA',
          'DISC FR BRAKE - 2SJ',
          'Disc FR Brake  _XBA',
          'Disc, Front Brake,YL8',
         'Disc,FR Brake (Cav 5&6  3&4)_ EFC',
          'Disc-FR-Brake 13" _Nissan',
          'Disc-FR-Brake 14" _Nissan (Cavity 7 & 8 )',
         'Disk FR Brake - Model L  YAD / YBA',
          'Disk FR Brake - YJC',
         'Disk FR Brake _ YHB',
          'Disk FR Brake _Vent -Y9T_Domestic',
          'Disk FR Brake _Vent -Y9T_Export',
         'Disk FR Brake _Vent _2VL',
          'Disk FR Brake, _Jazz (2CT)_Cav. 3 & 4',
          'Disk FR Brake_Vent - YRA',
         'Disk RR Brake _Solid - YRA',
          'Disk RR Brake_Solid - YAD',                                
          'Drum RR Brake  _XBA',
          'Drum RR Brake - YRA',
         'Drum RR Brake _ 2SJ',
        'Drum RR Brake _ 2UA',
          'Drum RR Brake _ XBA ABS',
          'Drum RR Brake _Y9T',
         'Drum Rear Brake (Cav 5 & 6)_YP8',
          'Drum Rear Brake _ Jazz (2CT) _ Cav. 3 & 4',
          'Flywheel _ Cav1 & 2_Diesel MT set#1',
         'NI2 DISC',
          'Y9T DISC'])]

group_high_b= dff_b.loc[dff_b['component_id'].isin(['Brake drum 14", Nissan set # 4      Cav. 7 & 8',
           'M FW J30'])]

group_low_fs = dff_fs.loc[dff_fs['component_id'].isin(['Brake Disc _Cav 7:&8: (Model K)',
          'Brake Disc, (Cav 7,8,9,10)Alto',
          'Crank Case',
        'Cylinder Block Cast_new pattern',
         'Cylinder Block _ BMG',
          'Cylinder Block _ CMA',
          'DISC FR BRAKE - 2SJ',
          'Disc FR Brake  _XBA',
          'Disc, Front Brake,YL8',
         'Disc,FR Brake (Cav 5&6  3&4)_ EFC',
          'Disc-FR-Brake 13" _Nissan',
          'Disc-FR-Brake 14" _Nissan (Cavity 7 & 8 )',
         'Disk FR Brake - Model L  YAD / YBA',
          'Disk FR Brake - YJC',
         'Disk FR Brake _ YHB',
          'Disk FR Brake _Vent -Y9T_Domestic',
          'Disk FR Brake _Vent -Y9T_Export',
         'Disk FR Brake _Vent _2VL',
          'Disk FR Brake, _Jazz (2CT)_Cav. 3 & 4',
          'Disk FR Brake_Vent - YRA',
         'Disk RR Brake _Solid - YRA',
          'Disk RR Brake_Solid - YAD',                                
          'Drum RR Brake  _XBA',
          'Drum RR Brake - YRA',
         'Drum RR Brake _ 2SJ',
        'Drum RR Brake _ 2UA',
          'Drum RR Brake _ XBA ABS',
          'Drum RR Brake _Y9T',
         'Drum Rear Brake (Cav 5 & 6)_YP8',
          'Drum Rear Brake _ Jazz (2CT) _ Cav. 3 & 4',
          'Flywheel _ Cav1 & 2_Diesel MT set#1',
         'NI2 DISC',
          'Y9T DISC'])]
group_high_fs = dff_fs.loc[dff_fs['component_id'].isin(['Brake drum 14", Nissan set # 4      Cav. 7 & 8',
           'M FW J30'])]

group_low = dfp.loc[dfp['component_id'].isin(['Brake Disc _Cav 7:&8: (Model K)',
          'Brake Disc, (Cav 7,8,9,10)Alto',
          'Crank Case',
        'Cylinder Block Cast_new pattern',
         'Cylinder Block _ BMG',
          'Cylinder Block _ CMA',
          'DISC FR BRAKE - 2SJ',
          'Disc FR Brake  _XBA',
          'Disc, Front Brake,YL8',
         'Disc,FR Brake (Cav 5&6  3&4)_ EFC',
          'Disc-FR-Brake 13" _Nissan',
          'Disc-FR-Brake 14" _Nissan (Cavity 7 & 8 )',
         'Disk FR Brake - Model L  YAD / YBA',
          'Disk FR Brake - YJC',
         'Disk FR Brake _ YHB',
          'Disk FR Brake _Vent -Y9T_Domestic',
          'Disk FR Brake _Vent -Y9T_Export',
         'Disk FR Brake _Vent _2VL',
          'Disk FR Brake, _Jazz (2CT)_Cav. 3 & 4',
          'Disk FR Brake_Vent - YRA',
         'Disk RR Brake _Solid - YRA',
          'Disk RR Brake_Solid - YAD',                                
          'Drum RR Brake  _XBA',
          'Drum RR Brake - YRA',
         'Drum RR Brake _ 2SJ',
        'Drum RR Brake _ 2UA',
          'Drum RR Brake _ XBA ABS',
          'Drum RR Brake _Y9T',
         'Drum Rear Brake (Cav 5 & 6)_YP8',
          'Drum Rear Brake _ Jazz (2CT) _ Cav. 3 & 4',
          'Flywheel _ Cav1 & 2_Diesel MT set#1',
         'NI2 DISC',
          'Y9T DISC'])]

group_high = dfp.loc[dfp['component_id'].isin(['Brake drum 14", Nissan set # 4      Cav. 7 & 8',
           'M FW J30'])]

group_high_b.drop(group_high_b.columns[[0,1,2,3]],axis=1,inplace=True)
group_high_fs.drop(group_high_fs.columns[[0,1,2,3]],axis=1,inplace=True)
group_high.drop(group_high.columns[[0,1,2,3]],axis=1,inplace=True)

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group_high_b:
    X=group_high_b.drop(i,axis=1) #removing the target variable
    y = group_high_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group_high:
    X=group_high.drop(i,axis=1)
    y = group_high[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
#similar to df1 and df2 , all the same steps are carried out for df3
for i in group_high_fs:
    X=group_high_fs.drop(i,axis=1)
    y = group_high_fs[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group High')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

group_low_b.drop(group_low_b.columns[[0,1,2,3]],axis=1,inplace=True)
group_low_fs.drop(group_low_fs.columns[[0,1,2,3]],axis=1,inplace=True)
group_low.drop(group_low.columns[[0,1,2,3]],axis=1,inplace=True)

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group_low_b:
    X=group_low_b.drop(i,axis=1) #removing the target variable
    y = group_low_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group_low:
    X=group_low.drop(i,axis=1)
    y = group_low[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
#similar to df1 and df2 , all the same steps are carried out for df3
for i in group_low_fs:
    X=group_low_fs.drop(i,axis=1)
    y = group_low_fs[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group Low:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)


db.columns = db.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
dfs.columns = dfs.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
dpr.columns = dpr.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
#prepared sand and bentonite
a=[]
for each in db.component_id.unique():
    a.append(db.loc[db.component_id==each,:])
#only prepared sand
b=[]
for each in dpr.component_id.unique():
    b.append(dpr.loc[dpr.component_id==each,:])
    
#prepared sand and fss
c=[]
for each in dfs.component_id.unique():
    c.append(dfs.loc[dfs.component_id==each,:])

data_bentonite = a[0],a[1],a[2],a[3],a[4],a[5],a[6],a[7],a[8],a[9],a[10],a[11],a[12],a[13],a[14],a[15],a[16],a[17],a[18],a[19],a[20],a[21],a[22],a[23],a[24],a[25],a[26],a[27],a[28],a[29],a[30],a[31],a[32],a[33]
data_sand = b[0],b[1],b[2],b[3],b[4],b[5],b[6],b[7],b[8],b[9],b[10],b[11],b[12],b[13],b[14],b[15],b[16],b[17],b[18],b[19],b[20],b[21],b[22],b[23],b[24],b[25],b[26],b[27],b[28],b[29],b[30],b[31],b[32],b[33]
data_fss = c[0],c[1],c[2],c[3],c[4],c[5],c[6],c[7],c[8],c[9],c[10],c[11],c[12],c[13],c[14],c[15],c[16],c[17],c[18],c[19],c[20],c[21],c[22],c[23],c[24],c[25],c[26],c[27],c[28],c[29],c[30],c[31],c[32],c[33]

#prepared sand and bentonite dataframes:
q0 = a[0].drop(a[0].columns[[0,1,2]],axis=1)
q1 = a[1].drop(a[1].columns[[0,1,2]],axis=1)
q2 = a[2].drop(a[2].columns[[0,1,2]],axis=1)
q3 = a[3].drop(a[3].columns[[0,1,2]],axis=1)
q4 = a[4].drop(a[4].columns[[0,1,2]],axis=1)
q5 = a[5].drop(a[5].columns[[0,1,2]],axis=1)
q6 = a[6].drop(a[6].columns[[0,1,2]],axis=1)
q7 = a[7].drop(a[7].columns[[0,1,2]],axis=1)
q8 = a[8].drop(a[8].columns[[0,1,2]],axis=1)
q9 = a[9].drop(a[9].columns[[0,1,2]],axis=1)
q10 = a[10].drop(a[10].columns[[0,1,2]],axis=1)
q11 = a[10].drop(a[11].columns[[0,1,2]],axis=1)
q12 = a[12].drop(a[12].columns[[0,1,2]],axis=1)
q13 = a[13].drop(a[13].columns[[0,1,2]],axis=1)
q14 = a[14].drop(a[14].columns[[0,1,2]],axis=1)
q15 = a[15].drop(a[15].columns[[0,1,2]],axis=1)
q16 = a[16].drop(a[16].columns[[0,1,2]],axis=1)
q17 = a[17].drop(a[17].columns[[0,1,2]],axis=1)
q18 = a[18].drop(a[18].columns[[0,1,2]],axis=1)
q19 = a[19].drop(a[19].columns[[0,1,2]],axis=1)
q20 = a[20].drop(a[20].columns[[0,1,2]],axis=1)
q21 = a[21].drop(a[21].columns[[0,1,2]],axis=1)
q22 = a[22].drop(a[22].columns[[0,1,2]],axis=1)
q23 = a[23].drop(a[23].columns[[0,1,2]],axis=1)
q24 = a[24].drop(a[24].columns[[0,1,2]],axis=1)
q25 = a[25].drop(a[25].columns[[0,1,2]],axis=1)
q26 = a[26].drop(a[26].columns[[0,1,2]],axis=1)
q27 = a[27].drop(a[27].columns[[0,1,2]],axis=1)
q28 = a[28].drop(a[28].columns[[0,1,2]],axis=1)
q29 = a[29].drop(a[29].columns[[0,1,2]],axis=1)
q30 = a[30].drop(a[30].columns[[0,1,2]],axis=1)
q31 = a[31].drop(a[31].columns[[0,1,2]],axis=1)
q32 = a[32].drop(a[32].columns[[0,1,2]],axis=1)
q33 = a[33].drop(a[33].columns[[0,1,2]],axis=1)

#Only prepared sand data:
r0 = b[0].drop(b[0].columns[[0,1,2]],axis=1)
r1 = b[1].drop(b[1].columns[[0,1,2]],axis=1)
r2 = b[2].drop(b[2].columns[[0,1,2]],axis=1)
r3 = b[3].drop(b[3].columns[[0,1,2]],axis=1)
r4 = b[4].drop(b[4].columns[[0,1,2]],axis=1)
r5 = b[5].drop(b[5].columns[[0,1,2]],axis=1)
r6 = b[6].drop(b[6].columns[[0,1,2]],axis=1)
r7 = b[7].drop(b[7].columns[[0,1,2]],axis=1)
r8 = b[8].drop(b[8].columns[[0,1,2]],axis=1)
r9 = b[9].drop(b[9].columns[[0,1,2]],axis=1)
r10 = b[10].drop(b[10].columns[[0,1,2]],axis=1)
r11 = b[10].drop(b[11].columns[[0,1,2]],axis=1)
r12 = b[12].drop(b[12].columns[[0,1,2]],axis=1)
r13 = b[13].drop(b[13].columns[[0,1,2]],axis=1)
r14 = b[14].drop(b[14].columns[[0,1,2]],axis=1)
r15 = b[15].drop(b[15].columns[[0,1,2]],axis=1)
r16 = b[16].drop(b[16].columns[[0,1,2]],axis=1)
r17 = b[17].drop(b[17].columns[[0,1,2]],axis=1)
r18 = b[18].drop(b[18].columns[[0,1,2]],axis=1)
r19 = b[19].drop(b[19].columns[[0,1,2]],axis=1)
r20 = b[20].drop(b[20].columns[[0,1,2]],axis=1)
r21 = b[21].drop(b[21].columns[[0,1,2]],axis=1)
r22 = b[22].drop(b[22].columns[[0,1,2]],axis=1)
r23 = b[23].drop(b[23].columns[[0,1,2]],axis=1)
r24 = b[24].drop(b[24].columns[[0,1,2]],axis=1)
r25 = b[25].drop(b[25].columns[[0,1,2]],axis=1)
r26 = b[26].drop(b[26].columns[[0,1,2]],axis=1)
r27 = b[27].drop(b[27].columns[[0,1,2]],axis=1)
r28 = b[28].drop(b[28].columns[[0,1,2]],axis=1)
r29 = b[29].drop(b[29].columns[[0,1,2]],axis=1)
r30 = b[30].drop(b[30].columns[[0,1,2]],axis=1)
r31 = b[31].drop(b[31].columns[[0,1,2]],axis=1)
r32 = b[32].drop(b[32].columns[[0,1,2]],axis=1)
r33 = b[33].drop(b[33].columns[[0,1,2]],axis=1)


#prepared sand and FSS data:
s0 = c[0].drop(c[0].columns[[0,1,2]],axis=1)
s1 = c[1].drop(c[1].columns[[0,1,2]],axis=1)
s2 = c[2].drop(c[2].columns[[0,1,2]],axis=1)
s3 = c[3].drop(c[3].columns[[0,1,2]],axis=1)
s4 = c[4].drop(c[4].columns[[0,1,2]],axis=1)
s5 = c[5].drop(c[5].columns[[0,1,2]],axis=1)
s6 = c[6].drop(c[6].columns[[0,1,2]],axis=1)
s7 = c[7].drop(c[7].columns[[0,1,2]],axis=1)
s8 = c[8].drop(c[8].columns[[0,1,2]],axis=1)
s9 = c[9].drop(c[9].columns[[0,1,2]],axis=1)
s10 = c[10].drop(c[10].columns[[0,1,2]],axis=1)
s11 = c[10].drop(c[11].columns[[0,1,2]],axis=1)
s12 = c[12].drop(c[12].columns[[0,1,2]],axis=1)
s13 = c[13].drop(c[13].columns[[0,1,2]],axis=1)
s14 = c[14].drop(c[14].columns[[0,1,2]],axis=1)
s15 = c[15].drop(c[15].columns[[0,1,2]],axis=1)
s16 = c[16].drop(c[16].columns[[0,1,2]],axis=1)
s17 = c[17].drop(c[17].columns[[0,1,2]],axis=1)
s18 = c[18].drop(c[18].columns[[0,1,2]],axis=1)
s19 = c[19].drop(c[19].columns[[0,1,2]],axis=1)
s20 = c[20].drop(c[20].columns[[0,1,2]],axis=1)
s21 = c[21].drop(c[21].columns[[0,1,2]],axis=1)
s22 = c[22].drop(c[22].columns[[0,1,2]],axis=1)
s23 = c[23].drop(c[23].columns[[0,1,2]],axis=1)
s24 = c[24].drop(c[24].columns[[0,1,2]],axis=1)
s25 = c[25].drop(c[25].columns[[0,1,2]],axis=1)
s26 = c[26].drop(c[26].columns[[0,1,2]],axis=1)
s27 = c[27].drop(c[27].columns[[0,1,2]],axis=1)
s28 = c[28].drop(c[28].columns[[0,1,2]],axis=1)
s29 = c[29].drop(c[29].columns[[0,1,2]],axis=1)
s30 = c[30].drop(c[30].columns[[0,1,2]],axis=1)
s31 = c[31].drop(c[31].columns[[0,1,2]],axis=1)
s32 = c[32].drop(c[32].columns[[0,1,2]],axis=1)
s33 = c[33].drop(c[33].columns[[0,1,2]],axis=1)


bentonite = [q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30,q31,q32,q33]
sand = [r0,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13,r14,r15,r16,r17,r18,r19,r20,r21,r22,r23,r24,r25,r26,r27,r28,r29,r30,r31,r32,r33]
fss = [s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33]

d2= d1[['gcs_', 'compactability_', 'active_clay_', 'wet_tensile_strength_', 'moisture_', 'permeability']]
d3 = d1[['loi_', 'inert_fines_', 'volatile_matter_']]
d4 = d1[['gfnafs']]

df_names = []

comp1={}
for j in bentonite:
    if len(j)<5:
        print(j['component_id'],':','Insufficient data to build a model')
        continue
    else:
        pass
        
        for i in j:
            if i!='component_id':
                X=j.drop([i,'component_id'],axis=1) 
                y = j[i]
                if i == 'loi_':
                    break 

                sc=StandardScaler()
                X=sc.fit_transform(X)
                X = pd.DataFrame(X)

                from sklearn.model_selection import KFold 
                from sklearn import metrics
                n_split = 5 
                kf=KFold(n_splits=n_split,shuffle=True,random_state=0) 

                for model,name in zip([boost_rf],['boosted_random_forest']):
                    rmse_test=[]
                    r2_test=[]
                    adj_r2_test=[]
                    rmse_train=[]
                    r2_train=[]
                    adj_r2_train=[]
                    
                    for train,test in kf.split(X,y):
                        X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                        y_train,y_test=y.iloc[train],y.iloc[test]
                        model.fit(X_train,y_train) #fitting the model
                        y_predict_test=model.predict(X_test) #predicting the test result
                        mse_test=round(metrics.mean_squared_error(y_test,y_predict_test),3) #calculating mse
                        rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                        rSquare_test = round(r2_score((y_test),(y_predict_test)),3) #calculating rsquared for test
                        adj_rsquare_test= round(1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for test
                        r2_test.append(rSquare_test) #appending the result to the empty list
                        adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list

                        y_predict_train=model.predict(X_train) #predicting the result for train
                        mse_train=round(metrics.mean_squared_error(y_train,y_predict_train),3) #calculating mse for train
                        rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                        rSquare_train = round(r2_score((y_train),(y_predict_train)),3) #calculating rsquared for train
                        adj_rsquare_train= round(1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for train
                        r2_train.append(rSquare_train) #appending the result to the empty list
                        adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                    exec(f"df_{name}_{i} = pd.DataFrame()")
                    exec(f"df_{name}_{i}['Name'] =[name]*n_split")
                    exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
                    exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
                    exec(f"df_{name}_{i}['r2_train'] =r2_train")
                    exec(f"df_{name}_{i}['r2_test'] =r2_test")
                    exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
                    exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
                    exec(f"df_{name}_{i}['output'] = [i]*n_split")
                    df_names.append(f"df_{name}_{i}")
            
            
    collection = pd.DataFrame() #collecting the results in dataframe

    for nn in df_names:
        exec(f"collection=pd.concat([collection,{nn}])")

        collection['new_name'] = collection['output']


    df4 = collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

    abc = list(d2.columns)

    new = pd.DataFrame(columns=df4.columns)
    for names in range(len(abc)):
        val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
        df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
        new=pd.concat([new,df5])           
    
    comp=j.component_id.unique()[0]
    comp1[comp]=new
    print('\n')
    print('\033[1m'+ comp+':')
    print('\n')
    new[['r2_train','r2_test','adj_r2_train','adj_r2_test']] = round(new[['r2_train','r2_test','adj_r2_train','adj_r2_test']]*100,2)
    new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
    print(new)
    print('\n')
    
df_names = []

comp1={}
for j in sand:
    if len(j)<5:
        print(j['component_id'],':','Insufficient data to build a model')
        continue
    else:
        pass
        
        for i in j:
            if i!='component_id':
                X=j.drop([i,'component_id'],axis=1) 
                y = j[i]
                if i == 'gcs_':
                    break 

                sc=StandardScaler()
                X=sc.fit_transform(X)
                X = pd.DataFrame(X)

                from sklearn.model_selection import KFold 
                from sklearn import metrics
                n_split = 5 
                kf=KFold(n_splits=n_split,shuffle=True,random_state=0) 

                for model,name in zip([boost_rf],['boosted_random_forest']):
                    rmse_test=[]
                    r2_test=[]
                    adj_r2_test=[]
                    rmse_train=[]
                    r2_train=[]
                    adj_r2_train=[]
                    
                    for train,test in kf.split(X,y):
                        X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                        y_train,y_test=y.iloc[train],y.iloc[test]
                        model.fit(X_train,y_train) #fitting the model
                        y_predict_test=model.predict(X_test) #predicting the test result
                        mse_test=round(metrics.mean_squared_error(y_test,y_predict_test),3) #calculating mse
                        rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                        rSquare_test = round(r2_score((y_test),(y_predict_test)),3) #calculating rsquared for test
                        adj_rsquare_test= round(1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for test
                        r2_test.append(rSquare_test) #appending the result to the empty list
                        adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list

                        y_predict_train=model.predict(X_train) #predicting the result for train
                        mse_train=round(metrics.mean_squared_error(y_train,y_predict_train),3) #calculating mse for train
                        rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                        rSquare_train = round(r2_score((y_train),(y_predict_train)),3) #calculating rsquared for train
                        adj_rsquare_train= round(1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for train
                        r2_train.append(rSquare_train) #appending the result to the empty list
                        adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                    exec(f"df_{name}_{i} = pd.DataFrame()")
                    exec(f"df_{name}_{i}['Name'] =[name]*n_split")
                    exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
                    exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
                    exec(f"df_{name}_{i}['r2_train'] =r2_train")
                    exec(f"df_{name}_{i}['r2_test'] =r2_test")
                    exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
                    exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
                    exec(f"df_{name}_{i}['output'] = [i]*n_split")
                    df_names.append(f"df_{name}_{i}")
            
            
    collection = pd.DataFrame() #collecting the results in dataframe

    for nn in df_names:
        exec(f"collection=pd.concat([collection,{nn}])")

        collection['new_name'] = collection['output']


    df4 = collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

    abc = list(d3.columns)

    new = pd.DataFrame(columns=df4.columns)
    for names in range(len(abc)):
        val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
        df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
        new=pd.concat([new,df5])           
    
    comp=j.component_id.unique()[0]
    comp1[comp]=new
    print('\n')
    print('\033[1m'+ comp+':')
    print('\n')
    new[['r2_train','r2_test','adj_r2_train','adj_r2_test']] = round(new[['r2_train','r2_test','adj_r2_train','adj_r2_test']]*100,2)
    new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
    print(new)
    print('\n')
    
df_names = []

comp1={}
for j in fss:
    if len(j)<5:
        print(j['component_id'],':','Insufficient data to build a model')
        continue
    else:
        pass
        
        for i in j:
            if i!='component_id':
                X=j.drop([i,'component_id'],axis=1) 
                y = j[i]
                if i == 'gcs_':
                    break 

                sc=StandardScaler()
                X=sc.fit_transform(X)
                X = pd.DataFrame(X)

                from sklearn.model_selection import KFold 
                from sklearn import metrics
                n_split = 5 
                kf=KFold(n_splits=n_split,shuffle=True,random_state=0) 

                for model,name in zip([boost_rf],['boosted_random_forest']):
                    rmse_test=[]
                    r2_test=[]
                    adj_r2_test=[]
                    rmse_train=[]
                    r2_train=[]
                    adj_r2_train=[]
                    
                    for train,test in kf.split(X,y):
                        X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                        y_train,y_test=y.iloc[train],y.iloc[test]
                        model.fit(X_train,y_train) #fitting the model
                        y_predict_test=model.predict(X_test) #predicting the test result
                        mse_test=round(metrics.mean_squared_error(y_test,y_predict_test),3) #calculating mse
                        rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                        rSquare_test = round(r2_score((y_test),(y_predict_test)),3) #calculating rsquared for test
                        adj_rsquare_test= round(1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for test
                        r2_test.append(rSquare_test) #appending the result to the empty list
                        adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list

                        y_predict_train=model.predict(X_train) #predicting the result for train
                        mse_train=round(metrics.mean_squared_error(y_train,y_predict_train),3) #calculating mse for train
                        rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                        rSquare_train = round(r2_score((y_train),(y_predict_train)),3) #calculating rsquared for train
                        adj_rsquare_train= round(1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for train
                        r2_train.append(rSquare_train) #appending the result to the empty list
                        adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                    exec(f"df_{name}_{i} = pd.DataFrame()")
                    exec(f"df_{name}_{i}['Name'] =[name]*n_split")
                    exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
                    exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
                    exec(f"df_{name}_{i}['r2_train'] =r2_train")
                    exec(f"df_{name}_{i}['r2_test'] =r2_test")
                    exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
                    exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
                    exec(f"df_{name}_{i}['output'] = [i]*n_split")
                    df_names.append(f"df_{name}_{i}")
            
            
    collection = pd.DataFrame() #collecting the results in dataframe

    for nn in df_names:
        exec(f"collection=pd.concat([collection,{nn}])")

        collection['new_name'] = collection['output']


    df4 = collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

    abc = list(d4.columns)

    new = pd.DataFrame(columns=df4.columns)
    for names in range(len(abc)):
        val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
        df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
        new=pd.concat([new,df5])           
    
    comp=j.component_id.unique()[0]
    comp1[comp]=new
    print('\n')
    print('\033[1m'+ comp+':')
    print('\n')
    new[['r2_train','r2_test','adj_r2_train','adj_r2_test']] = round(new[['r2_train','r2_test','adj_r2_train','adj_r2_test']]*100,2)
    new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
    print(new)
    print('\n')

All
                       rmse_train  rmse_test  r_squared_train  r_squared_test  \
gcs_                         0.03       0.09            94.66           40.86   
compactability_              0.34       1.30            98.24           74.84   
active_clay_                 0.00       0.02           100.00           98.40   
wet_tensile_strength_        0.63       2.21            94.27           29.49   
moisture_                    0.03       0.08            97.28           71.29   
permeability                 1.33       4.75            97.67           70.18   
loi_                         0.00       0.04            99.94           92.40   
inert_fines_                 0.00       0.02            99.99           97.90   
volatile_matter_             0.00       0.03            99.88           85.19   
gfnafs                       0.02       0.15            99.97           98.55   

                       adj_r2_train  adj_r2_test  
gcs_                          94.63        40.56  
co

C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Group High
                       rmse_train  rmse_test  r_squared_train  r_squared_test  \
gcs_                         0.03       0.10            93.13           31.07   
compactability_              0.37       1.38            97.54           65.81   
active_clay_                 0.01       0.04            99.87           88.93   
wet_tensile_strength_        0.71       2.46            93.49           19.96   
moisture_                    0.02       0.08            97.80           69.59   
permeability                 1.50       5.39            96.61           55.82   
loi_                         0.01       0.06            99.54           79.88   
inert_fines_                 0.01       0.06            99.76           80.46   
volatile_matter_             0.01       0.04            99.34           76.31   
gfnafs                       0.11       0.49            99.19           83.14   

                       adj_r2_train  adj_r2_test  
gcs_                          92.87        28.

C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Group Low:
                       rmse_train  rmse_test  r_squared_train  r_squared_test  \
gcs_                         0.03       0.09            94.61           38.50   
compactability_              0.34       1.31            98.31           75.14   
active_clay_                 0.00       0.02            99.99           98.30   
wet_tensile_strength_        0.63       2.22            94.26           27.56   
moisture_                    0.03       0.08            97.27           71.32   
permeability                 1.34       4.80            97.68           70.14   
loi_                         0.00       0.04            99.93           92.68   
inert_fines_                 0.00       0.02            99.99           98.11   
volatile_matter_             0.00       0.03            99.87           85.19   
gfnafs                       0.03       0.15            99.95           98.43   

                       adj_r2_train  adj_r2_test  
gcs_                          94.58        38.

...



DISC FR BRAKE - 2SJ:


        adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  rmse_train
gfnafs         -inf          -inf  -174.76     85.62       0.69        0.25




Disk FR Brake _Vent -Y9T_Domestic:


        adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  rmse_train
gfnafs       -46.36         95.08     0.04     96.64       1.08        0.25




Disc-FR-Brake 13" _Nissan:


        adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  rmse_train
gfnafs       -263.9          87.4    -1.08      96.5       0.76        0.15




Drum Rear Brake (Cav 5 & 6)_YP8:


        adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  rmse_train
gfnafs       824.78        169.18   -31.78     87.42       0.89        0.41




Drum Rear Brake _ Jazz (2CT) _ Cav. 3 & 4:


        adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  rmse_train
gfnafs        21.98         95.36     32.9      96.0       0.97        0.25




Cylinder Block _ CMA:


        adj_r2_test  adj_

C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)




Disk FR Brake _Vent -Y9T_Export:


        adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  rmse_train
gfnafs        129.7         143.1     52.4     31.06       0.74        0.61




Disk RR Brake _Solid - YRA:


        adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  rmse_train
gfnafs        -3.96         94.06     44.3     96.82       0.95        0.24




Disc, Front Brake,YL8:


        adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  rmse_train
gfnafs        -24.1         96.38    40.44     98.26       0.69        0.14




C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)




Drum RR Brake _ 2SJ:


        adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  rmse_train
gfnafs       418.53         178.1  -173.03     33.04       1.43        1.06




Disk FR Brake - Model L  YAD / YBA:


        adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  rmse_train
gfnafs        19.78         96.54    35.58     97.22       0.74        0.17




NI2 DISC:


        adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  rmse_train
gfnafs      5530.82         159.9  -2313.7     73.38       1.02         0.7




Flywheel _ Cav1 & 2_Diesel MT set#1:


        adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  rmse_train
gfnafs     30905.38         144.8 -9141.62     86.56       0.33         0.2


2367    Cylinder Block Cast_new pattern
Name: component_id, dtype: object : Insufficient data to build a model
3344    HONDA 2LT DRUM
Name: component_id, dtype: object : Insufficient data to build a model


# Foundry name : Brakes

In [1]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
#loading the prepared sand data
dfp = pd.read_excel('Preparedsand_01-Jan-2019_TO_15-Mar-2020_Line E.xlsx',skiprows=[0,1,2,3,4])
dfp.drop(dfp.columns[[3, 14, 27]], axis = 1, inplace = True)#removing heat number, specimen weight and relative humidity
dfp = dfp.dropna(axis=1,how='all') #removing the columns with full null values
dfp.fillna(dfp.median(), inplace=True)  #replacing rest of the null values with median values
dfb = pd.read_excel('Bentonite_01-Jan-2019_TO_15-Mar-2020_Line E.xlsx',skiprows=[0,1,2,3,4])
dfb.drop(dfb.columns[[1, 2, 3, 4]], axis = 1, inplace = True)#removing batch no, supplier, quantity, and grade columns
dfb.columns = dfb.columns.str.replace(' ','_').str.replace('(','').str.replace(')','').str.replace('%','_raw').str.replace('kg/cm2','_raw').str.replace('gm/cm2','_raw')
dfb = dfb.dropna(axis=1,how='all') #removing the columns with full null values
dff_b = pd.merge(dfp,dfb,on='Date',how='left')
dff_b = dff_b.dropna(axis=0, how='all', subset=['Component Id'])
dff_b.fillna(method='ffill',inplace=True)
dff_b.fillna(method='bfill',inplace=True)
cols = list(dff_b)
cols[0],cols[1],cols[2],cols[3],cols[4],cols[5],cols[6],cols[7],cols[8],cols[9],cols[10],cols[11],cols[12],cols[13],cols[14],cols[15],cols[16],cols[17],cols[18],cols[19],cols[20],cols[21],cols[22] = cols[0],cols[1],cols[2],cols[3],cols[4],cols[5],cols[6],cols[7],cols[9],cols[13],cols[14],cols[8],cols[10],cols[11],cols[12],cols[15],cols[16],cols[17],cols[18],cols[19],cols[20],cols[21],cols[22]
dff_b = dff_b.loc[:,cols]
df1 = dff_b.copy()
db = dff_b.copy()
df1.drop(df1.columns[[0,1,2,3]],axis=1,inplace=True)
cols = list(dfp)
cols[0],cols[1],cols[2],cols[3],cols[4],cols[5],cols[6],cols[7],cols[8],cols[9],cols[10],cols[11],cols[12],cols[13],cols[14],cols[15],cols[16],cols[17] = cols[0],cols[1],cols[2],cols[3],cols[8],cols[11],cols[12],cols[4],cols[5],cols[6],cols[7],cols[9],cols[10],cols[13],cols[14],cols[15],cols[16],cols[17]
dfp = dfp.loc[:,cols]
dfp = dfp.dropna(axis=0, how='all', subset=['Component Id'])
df2 = dfp.copy()
dpr = dfp.copy()
df2.drop(df2.columns[[0,1,2,3]],axis=1,inplace=True)
#df1 has prepared sand and bentonite data
#df2 has only the prepared sand data
#df3 has prepared sand and FSS data
d1 = df1.copy()
d1.drop(d1.columns[[8, 11,12,13,14,15,16,17,18]], axis = 1, inplace = True)
rf = RandomForestRegressor(n_estimators=100,random_state=0)
boost_rf = AdaBoostRegressor(base_estimator = rf, n_estimators=50,random_state=0)
df1.columns = df1.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
df2.columns = df2.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
d1.columns = d1.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in df1:
    X=df1.drop(i,axis=1) #removing the target variable
    y = df1[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in df2:
    X=df2.drop(i,axis=1)
    y = df2[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'All')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

dff_b.columns = dff_b.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
dfp.columns = dfp.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')

group1_b= dff_b.loc[dff_b['component_id'].isin(['5110','5113','5158','5180','5181','5256','5257','7366','7367',
                                                '7426','7426-6','7427','7427-6','7427-7'])]
group1_ps = dfp.loc[dfp['component_id'].isin(['5110','5113','5158','5180','5181','5256','5257','7366','7367',
                                                '7426','7426-6','7427','7427-6','7427-7'])]

group1_b.drop(group1_b.columns[[0,1,2,3]],axis=1,inplace=True)
group1_ps.drop(group1_ps.columns[[0,1,2,3]],axis=1,inplace=True)

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group1_b:
    X=group1_b.drop(i,axis=1) #removing the target variable
    y = group1_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group1_ps:
    X=group1_ps.drop(i,axis=1)
    y = group1_ps[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group 1:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

group2_b= dff_b.loc[dff_b['component_id'].isin(['5025','5025/25','5025/26','5026','5099','5131','5140/41','5209','5243','5243/44',
                                                '5259','5259/60','5260','5304','5326','5326/27','5327','5328','5328/29',
                                                '5329','5344','5344/45','5345','5349','5349/50','5350','7025','7026',
                                                '7042','7043','7153','7162','7164','7180','7183','7220','7278','7279',
                                                '7654','7952'])]

group2_ps = dfp.loc[dfp['component_id'].isin(['5025','5025/25','5025/26','5026','5099','5131','5140/41','5209','5243','5243/44',
                                                '5259','5259/60','5260','5304','5326','5326/27','5327','5328','5328/29',
                                                '5329','5344','5344/45','5345','5349','5349/50','5350','7025','7026',
                                                '7042','7043','7153','7162','7164','7180','7183','7220','7278','7279',
                                                '7654','7952'])]
group2_b.drop(group2_b.columns[[0,1,2,3]],axis=1,inplace=True)
group2_ps.drop(group2_ps.columns[[0,1,2,3]],axis=1,inplace=True)


df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group2_b:
    X=group2_b.drop(i,axis=1) #removing the target variable
    y = group2_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group2_ps:
    X=group2_ps.drop(i,axis=1)
    y = group2_ps[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group 2:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

group3_b= dff_b.loc[dff_b['component_id'].isin(['5071','5072','5134','5167','5168','5291','9503'])]

group3_ps = dfp.loc[dfp['component_id'].isin(['5071','5072','5134','5167','5168','5291','9503'])]

group3_b.drop(group3_b.columns[[0,1,2,3]],axis=1,inplace=True)
group3_ps.drop(group3_ps.columns[[0,1,2,3]],axis=1,inplace=True)

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group3_b:
    X=group3_b.drop(i,axis=1) #removing the target variable
    y = group3_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group3_ps:
    X=group3_ps.drop(i,axis=1)
    y = group3_ps[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group 3:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

group4_b= dff_b.loc[dff_b['component_id'].isin(['5147','5346','5346/47','5347','5356','6810','6811','7774','7775',
                                                '7791','7792','7931','7932','7964','7964/65','7965'])]

group4_ps = dfp.loc[dfp['component_id'].isin(['5147','5346','5346/47','5347','5356','6810','6811','7774','7775',
                                                '7791','7792','7931','7932','7964','7964/65','7965'])]

group4_b.drop(group4_b.columns[[0,1,2,3]],axis=1,inplace=True)
group4_ps.drop(group4_ps.columns[[0,1,2,3]],axis=1,inplace=True)

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group4_b:
    X=group4_b.drop(i,axis=1) #removing the target variable
    y = group4_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group4_ps:
    X=group4_ps.drop(i,axis=1)
    y = group4_ps[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group 4:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

group5_b= dff_b.loc[dff_b['component_id'].isin(['5169','5183','5223','5224','5276','5276/77','5277','5313','5330',
                                                '5360','5361','5362','6582','7007','7007/47','7008','7008/47',
                                                '7009','7021','7022','7023','7024','7044','7045','7046','7046/47',
                                                '7047','7064','7065','7066','7329'])]

group5_ps = dfp.loc[dfp['component_id'].isin(['5169','5183','5223','5224','5276','5276/77','5277','5313','5330',
                                                '5360','5361','5362','6582','7007','7007/47','7008','7008/47',
                                                '7009','7021','7022','7023','7024','7044','7045','7046','7046/47',
                                                '7047','7064','7065','7066','7329'])]


group5_b.drop(group5_b.columns[[0,1,2,3]],axis=1,inplace=True)
group5_ps.drop(group5_ps.columns[[0,1,2,3]],axis=1,inplace=True)

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group5_b:
    X=group5_b.drop(i,axis=1) #removing the target variable
    y = group5_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group5_ps:
    X=group5_ps.drop(i,axis=1)
    y = group5_ps[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group 5:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

group6_b= dff_b.loc[dff_b['component_id'].isin(['5303','5351','6297','6310','6311','6825','6869','7054','7054/55',
                                                '7055','7056','7056/57','7057','7059','7059/60','7060','7282',
                                                '7284','7285','7368','7776','7777','7950','7950/51','7951'])]

group6_ps = dfp.loc[dfp['component_id'].isin(['5303','5351','6297','6310','6311','6825','6869','7054','7054/55',
                                                '7055','7056','7056/57','7057','7059','7059/60','7060','7282',
                                                '7284','7285','7368','7776','7777','7950','7950/51','7951'])]

group6_b.drop(group6_b.columns[[0,1,2,3]],axis=1,inplace=True)
group6_ps.drop(group6_ps.columns[[0,1,2,3]],axis=1,inplace=True)

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group6_b:
    X=group6_b.drop(i,axis=1) #removing the target variable
    y = group6_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group6_ps:
    X=group6_ps.drop(i,axis=1)
    y = group6_ps[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group 6:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

group4a_b= dff_b.loc[dff_b['component_id'].isin(['7470-4','7470-5','7471-4','7471-5'])]

group4a_ps = dfp.loc[dfp['component_id'].isin(['7470-4','7470-5','7471-4','7471-5'])]

group4a_b.drop(group4a_b.columns[[0,1,2,3]],axis=1,inplace=True)
group4a_ps.drop(group4a_ps.columns[[0,1,2,3]],axis=1,inplace=True)


df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group4a_b:
    X=group4a_b.drop(i,axis=1) #removing the target variable
    y = group4a_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group4a_ps:
    X=group4a_ps.drop(i,axis=1)
    y = group4a_ps[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group 4A:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

groupdev_b= dff_b.loc[dff_b['component_id'].isin(['5020','5029','5146','5191','5243/44/20','5244','5266','5292',
                                                  '5293','5299','5300','5307','5307/08','5308','6600','6981','6982',
                                                  '6987','6988','6989','6999','7003','7038','7039','7050/51','7158',
                                                  '7184','7203','7229','7249','7393','7411','7412','7471','7491',
                                                  '7552','RPT 77','RPT 82','RPT-7184'])]

groupdev_ps = dfp.loc[dfp['component_id'].isin(['5020','5029','5146','5191','5243/44/20','5244','5266','5292',
                                                  '5293','5299','5300','5307','5307/08','5308','6600','6981','6982',
                                                  '6987','6988','6989','6999','7003','7038','7039','7050/51','7158',
                                                  '7184','7203','7229','7249','7393','7411','7412','7471','7491',
                                                  '7552','RPT 77','RPT 82','RPT-7184'])]


groupdev_b.drop(groupdev_b.columns[[0,1,2,3]],axis=1,inplace=True)
groupdev_ps.drop(groupdev_ps.columns[[0,1,2,3]],axis=1,inplace=True)

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in groupdev_b:
    X=groupdev_b.drop(i,axis=1) #removing the target variable
    y = groupdev_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in groupdev_ps:
    X=groupdev_ps.drop(i,axis=1)
    y = groupdev_ps[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group Development:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

db.columns = db.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
dpr.columns = dpr.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')

dpr = dpr.dropna(axis=0, how='all', subset=['component_id'])

#prepared sand and bentonite
a=[]
for each in db.component_id.unique():
    a.append(db.loc[db.component_id==each,:])
#only prepared sand
b=[]
for each in dpr.component_id.unique():
    b.append(dpr.loc[dpr.component_id==each,:])
    
data_bentonite = a[0],a[1],a[2],a[3],a[4],a[5],a[6],a[7],a[8],a[9],a[10],a[11],a[12],a[13],a[14],a[15],a[16],a[17],a[18],a[19],a[20],a[21],a[22],a[23],a[24],a[25],a[26],a[27],a[28],a[29],a[30],a[31],a[32],a[33],a[34],a[35],a[36],a[37],a[38],a[39],a[40],a[41],a[42],a[43],a[44],a[45],a[46],a[47],a[48],a[49],a[50],a[51],a[52],a[53],a[54],a[55],a[56],a[57],a[58],a[59],a[60],a[61],a[62],a[63],a[64],a[65],a[66],a[67],a[68],a[69],a[70],a[71],a[72],a[73],a[74],a[75],a[76],a[77],a[78],a[79],a[80],a[81],a[82],a[83],a[84],a[85],a[86],a[87],a[88],a[89],a[90],a[91],a[92],a[93],a[94],a[95],a[96],a[97],a[98],a[99],a[100],a[101],a[102],a[103],a[104],a[105],a[106],a[107],a[108],a[109],a[110],a[111],a[112],a[113],a[114],a[115]
data_sand = b[0],b[1],b[2],b[3],b[4],b[5],b[6],b[7],b[8],b[9],b[10],b[11],b[12],b[13],b[14],b[15],b[16],b[17],b[18],b[19],b[20],b[21],b[22],b[23],b[24],b[25],b[26],b[27],b[28],b[29],b[30],b[31],b[32],b[33],b[34],b[35],b[36],b[37],b[38],b[39],b[40],b[41],b[42],b[43],b[44],b[45],b[46],b[47],b[48],b[49],b[50],b[51],b[52],b[53],b[54],b[55],b[56],b[57],b[58],b[59],b[60],b[61],b[62],b[63],b[64],b[65],b[66],b[67],b[68],b[69],b[70],b[71],b[72],b[73],b[74],b[75],b[76],b[77],b[78],b[79],b[80],b[81],b[82],b[83],b[84],b[85],b[86],b[87],b[88],b[89],b[90],b[91],b[92],b[93],b[94],b[95],b[96],b[97],b[98],b[99],b[100],b[101],b[102],b[103],b[104],b[105],b[106],b[107],b[108],b[109],b[110],b[111],b[112],b[113],b[114],b[115]

#prepared sand and bentonite dataframes:
q0 = a[0].drop(a[0].columns[[0,1,2]],axis=1)
q1 = a[1].drop(a[1].columns[[0,1,2]],axis=1)
q2 = a[2].drop(a[2].columns[[0,1,2]],axis=1)
q3 = a[3].drop(a[3].columns[[0,1,2]],axis=1)
q4 = a[4].drop(a[4].columns[[0,1,2]],axis=1)
q5 = a[5].drop(a[5].columns[[0,1,2]],axis=1)
q6 = a[6].drop(a[6].columns[[0,1,2]],axis=1)
q7 = a[7].drop(a[7].columns[[0,1,2]],axis=1)
q8 = a[8].drop(a[8].columns[[0,1,2]],axis=1)
q9 = a[9].drop(a[9].columns[[0,1,2]],axis=1)
q10 = a[10].drop(a[10].columns[[0,1,2]],axis=1)
q11 = a[11].drop(a[11].columns[[0,1,2]],axis=1)
q12 = a[12].drop(a[12].columns[[0,1,2]],axis=1)
q13 = a[13].drop(a[13].columns[[0,1,2]],axis=1)
q14 = a[14].drop(a[14].columns[[0,1,2]],axis=1)
q15 = a[15].drop(a[15].columns[[0,1,2]],axis=1)
q16 = a[16].drop(a[16].columns[[0,1,2]],axis=1)
q17 = a[17].drop(a[17].columns[[0,1,2]],axis=1)
q18 = a[18].drop(a[18].columns[[0,1,2]],axis=1)
q19 = a[19].drop(a[19].columns[[0,1,2]],axis=1)
q20 = a[20].drop(a[20].columns[[0,1,2]],axis=1)
q21 = a[21].drop(a[21].columns[[0,1,2]],axis=1)
q22 = a[22].drop(a[22].columns[[0,1,2]],axis=1)
q23 = a[23].drop(a[23].columns[[0,1,2]],axis=1)
q24 = a[24].drop(a[24].columns[[0,1,2]],axis=1)
q25 = a[25].drop(a[25].columns[[0,1,2]],axis=1)
q26 = a[26].drop(a[26].columns[[0,1,2]],axis=1)
q27 = a[27].drop(a[27].columns[[0,1,2]],axis=1)
q28 = a[28].drop(a[28].columns[[0,1,2]],axis=1)
q29 = a[29].drop(a[29].columns[[0,1,2]],axis=1)
q30 = a[30].drop(a[30].columns[[0,1,2]],axis=1)
q31 = a[31].drop(a[31].columns[[0,1,2]],axis=1)
q32 = a[32].drop(a[32].columns[[0,1,2]],axis=1)
q33 = a[33].drop(a[33].columns[[0,1,2]],axis=1)
q34 = a[34].drop(a[34].columns[[0,1,2]],axis=1)
q35 = a[35].drop(a[35].columns[[0,1,2]],axis=1)
q36 = a[36].drop(a[36].columns[[0,1,2]],axis=1)
q37 = a[37].drop(a[37].columns[[0,1,2]],axis=1)
q38 = a[38].drop(a[38].columns[[0,1,2]],axis=1)
q39 = a[39].drop(a[39].columns[[0,1,2]],axis=1)
q40 = a[40].drop(a[40].columns[[0,1,2]],axis=1)
q41 = a[41].drop(a[41].columns[[0,1,2]],axis=1)
q42 = a[42].drop(a[42].columns[[0,1,2]],axis=1)
q43 = a[43].drop(a[43].columns[[0,1,2]],axis=1)
q44 = a[44].drop(a[44].columns[[0,1,2]],axis=1)
q45 = a[45].drop(a[45].columns[[0,1,2]],axis=1)
q46 = a[46].drop(a[46].columns[[0,1,2]],axis=1)
q47 = a[47].drop(a[47].columns[[0,1,2]],axis=1)
q48 = a[48].drop(a[48].columns[[0,1,2]],axis=1)
q49 = a[49].drop(a[49].columns[[0,1,2]],axis=1)
q50 = a[50].drop(a[50].columns[[0,1,2]],axis=1)
q51 = a[51].drop(a[51].columns[[0,1,2]],axis=1)
q52 = a[52].drop(a[52].columns[[0,1,2]],axis=1)
q53 = a[53].drop(a[53].columns[[0,1,2]],axis=1)
q54 = a[54].drop(a[54].columns[[0,1,2]],axis=1)
q55 = a[55].drop(a[55].columns[[0,1,2]],axis=1)
q56 = a[56].drop(a[56].columns[[0,1,2]],axis=1)
q57 = a[57].drop(a[57].columns[[0,1,2]],axis=1)
q58 = a[58].drop(a[58].columns[[0,1,2]],axis=1)
q59 = a[59].drop(a[59].columns[[0,1,2]],axis=1)
q60 = a[60].drop(a[60].columns[[0,1,2]],axis=1)
q61 = a[61].drop(a[61].columns[[0,1,2]],axis=1)
q62 = a[62].drop(a[62].columns[[0,1,2]],axis=1)
q63 = a[63].drop(a[63].columns[[0,1,2]],axis=1)
q64 = a[64].drop(a[64].columns[[0,1,2]],axis=1)
q65 = a[65].drop(a[65].columns[[0,1,2]],axis=1)
q66 = a[66].drop(a[66].columns[[0,1,2]],axis=1)
q67 = a[67].drop(a[67].columns[[0,1,2]],axis=1)
q68 = a[68].drop(a[68].columns[[0,1,2]],axis=1)
q69 = a[69].drop(a[69].columns[[0,1,2]],axis=1)
q70 = a[70].drop(a[70].columns[[0,1,2]],axis=1)
q71 = a[71].drop(a[71].columns[[0,1,2]],axis=1)
q72= a[72].drop(a[72].columns[[0,1,2]],axis=1)
q73 = a[73].drop(a[73].columns[[0,1,2]],axis=1)
q74 = a[74].drop(a[74].columns[[0,1,2]],axis=1)
q75= a[75].drop(a[75].columns[[0,1,2]],axis=1)
q76 = a[76].drop(a[76].columns[[0,1,2]],axis=1)
q77 = a[77].drop(a[77].columns[[0,1,2]],axis=1)##
q78 = a[78].drop(a[78].columns[[0,1,2]],axis=1)
q79 = a[79].drop(a[79].columns[[0,1,2]],axis=1)
q80 = a[80].drop(a[80].columns[[0,1,2]],axis=1)
q81 = a[81].drop(a[81].columns[[0,1,2]],axis=1)
q82 = a[82].drop(a[82].columns[[0,1,2]],axis=1)
q83 = a[83].drop(a[83].columns[[0,1,2]],axis=1)
q84 = a[84].drop(a[84].columns[[0,1,2]],axis=1)
q85 = a[85].drop(a[85].columns[[0,1,2]],axis=1)
q86 = a[86].drop(a[86].columns[[0,1,2]],axis=1)
q87 = a[87].drop(a[87].columns[[0,1,2]],axis=1)
q88 = a[88].drop(a[88].columns[[0,1,2]],axis=1)
q89 = a[89].drop(a[89].columns[[0,1,2]],axis=1)
q90 = a[90].drop(a[90].columns[[0,1,2]],axis=1)
q91 = a[91].drop(a[91].columns[[0,1,2]],axis=1)
q92 = a[92].drop(a[92].columns[[0,1,2]],axis=1)
q93 = a[93].drop(a[93].columns[[0,1,2]],axis=1)
q94 = a[94].drop(a[94].columns[[0,1,2]],axis=1)
q95 = a[95].drop(a[95].columns[[0,1,2]],axis=1)
q96 = a[96].drop(a[96].columns[[0,1,2]],axis=1)
q97 = a[97].drop(a[97].columns[[0,1,2]],axis=1)
q98 = a[98].drop(a[98].columns[[0,1,2]],axis=1)
q99 = a[99].drop(a[99].columns[[0,1,2]],axis=1)
q100 = a[100].drop(a[100].columns[[0,1,2]],axis=1)
q101= a[101].drop(a[101].columns[[0,1,2]],axis=1)
q102= a[102].drop(a[102].columns[[0,1,2]],axis=1)
q103= a[103].drop(a[103].columns[[0,1,2]],axis=1)
q104= a[104].drop(a[104].columns[[0,1,2]],axis=1)
q105= a[105].drop(a[105].columns[[0,1,2]],axis=1)
q106= a[106].drop(a[106].columns[[0,1,2]],axis=1)
q107= a[107].drop(a[107].columns[[0,1,2]],axis=1)
q108= a[108].drop(a[108].columns[[0,1,2]],axis=1)
q109= a[109].drop(a[109].columns[[0,1,2]],axis=1)
q110= a[110].drop(a[110].columns[[0,1,2]],axis=1)
q111= a[111].drop(a[111].columns[[0,1,2]],axis=1)
q112= a[112].drop(a[112].columns[[0,1,2]],axis=1)
q113= a[113].drop(a[113].columns[[0,1,2]],axis=1)
q114 = a[114].drop(a[114].columns[[0,1,2]],axis=1)
q115 = a[115].drop(a[115].columns[[0,1,2]],axis=1)


#only prepared sand data:
r0 = b[0].drop(b[0].columns[[0,1,2]],axis=1)
r1 = b[1].drop(b[1].columns[[0,1,2]],axis=1)
r2 = b[2].drop(b[2].columns[[0,1,2]],axis=1)
r3 = b[3].drop(b[3].columns[[0,1,2]],axis=1)
r4 = b[4].drop(b[4].columns[[0,1,2]],axis=1)
r5 = b[5].drop(b[5].columns[[0,1,2]],axis=1)
r6 = b[6].drop(b[6].columns[[0,1,2]],axis=1)
r7 = b[7].drop(b[7].columns[[0,1,2]],axis=1)
r8 = b[8].drop(b[8].columns[[0,1,2]],axis=1)
r9 = b[9].drop(b[9].columns[[0,1,2]],axis=1)
r10 = b[10].drop(b[10].columns[[0,1,2]],axis=1)
r11 = b[11].drop(b[11].columns[[0,1,2]],axis=1)
r12 = b[12].drop(b[12].columns[[0,1,2]],axis=1)
r13 = b[13].drop(b[13].columns[[0,1,2]],axis=1)
r14 = b[14].drop(b[14].columns[[0,1,2]],axis=1)
r15 = b[15].drop(b[15].columns[[0,1,2]],axis=1)
r16 = b[16].drop(b[16].columns[[0,1,2]],axis=1)
r17 = b[17].drop(b[17].columns[[0,1,2]],axis=1)
r18 = b[18].drop(b[18].columns[[0,1,2]],axis=1)
r19 = b[19].drop(b[19].columns[[0,1,2]],axis=1)
r20 = b[20].drop(b[20].columns[[0,1,2]],axis=1)
r21 = b[21].drop(b[21].columns[[0,1,2]],axis=1)
r22 = b[22].drop(b[22].columns[[0,1,2]],axis=1)
r23 = b[23].drop(b[23].columns[[0,1,2]],axis=1)
r24 = b[24].drop(b[24].columns[[0,1,2]],axis=1)
r25 = b[25].drop(b[25].columns[[0,1,2]],axis=1)
r26 = b[26].drop(b[26].columns[[0,1,2]],axis=1)
r27 = b[27].drop(b[27].columns[[0,1,2]],axis=1)
r28 = b[28].drop(b[28].columns[[0,1,2]],axis=1)
r29 = b[29].drop(b[29].columns[[0,1,2]],axis=1)
r30 = b[30].drop(b[30].columns[[0,1,2]],axis=1)
r31 = b[31].drop(b[31].columns[[0,1,2]],axis=1)
r32 = b[32].drop(b[32].columns[[0,1,2]],axis=1)
r33 = b[33].drop(b[33].columns[[0,1,2]],axis=1)
r34 = b[34].drop(b[34].columns[[0,1,2]],axis=1)
r35 = b[35].drop(b[35].columns[[0,1,2]],axis=1)
r36 = b[36].drop(b[36].columns[[0,1,2]],axis=1)
r37 = b[37].drop(b[37].columns[[0,1,2]],axis=1)
r38 = b[38].drop(b[38].columns[[0,1,2]],axis=1)
r39 = b[39].drop(b[39].columns[[0,1,2]],axis=1)
r40 = b[40].drop(b[40].columns[[0,1,2]],axis=1)
r41 = b[41].drop(b[41].columns[[0,1,2]],axis=1)
r42 = b[42].drop(b[42].columns[[0,1,2]],axis=1)
r43 = b[43].drop(b[43].columns[[0,1,2]],axis=1)
r44 = b[44].drop(b[44].columns[[0,1,2]],axis=1)
r45 = b[45].drop(b[45].columns[[0,1,2]],axis=1)
r46 = b[46].drop(b[46].columns[[0,1,2]],axis=1)
r47 = b[47].drop(b[47].columns[[0,1,2]],axis=1)
r48 = b[48].drop(b[48].columns[[0,1,2]],axis=1)
r49 = b[49].drop(b[49].columns[[0,1,2]],axis=1)
r50 = b[50].drop(b[50].columns[[0,1,2]],axis=1)
r51 = b[51].drop(b[51].columns[[0,1,2]],axis=1)
r52 = b[52].drop(b[52].columns[[0,1,2]],axis=1)
r53 = b[53].drop(b[53].columns[[0,1,2]],axis=1)
r54 = b[54].drop(b[54].columns[[0,1,2]],axis=1)
r55 = b[55].drop(b[55].columns[[0,1,2]],axis=1)
r56 = b[56].drop(b[56].columns[[0,1,2]],axis=1)
r57 = b[57].drop(b[57].columns[[0,1,2]],axis=1)
r58 = b[58].drop(b[58].columns[[0,1,2]],axis=1)
r59 = b[59].drop(b[59].columns[[0,1,2]],axis=1)
r60 = b[60].drop(b[60].columns[[0,1,2]],axis=1)
r61 = b[61].drop(b[61].columns[[0,1,2]],axis=1)
r62 = b[62].drop(b[62].columns[[0,1,2]],axis=1)
r63 = b[63].drop(b[63].columns[[0,1,2]],axis=1)
r64 = b[64].drop(b[64].columns[[0,1,2]],axis=1)
r65 = b[65].drop(b[65].columns[[0,1,2]],axis=1)
r66 = b[66].drop(b[66].columns[[0,1,2]],axis=1)
r67 = b[67].drop(b[67].columns[[0,1,2]],axis=1)
r68 = b[68].drop(b[68].columns[[0,1,2]],axis=1)
r69 = b[69].drop(b[69].columns[[0,1,2]],axis=1)
r70 = b[70].drop(b[70].columns[[0,1,2]],axis=1)
r71 = b[71].drop(b[71].columns[[0,1,2]],axis=1)
r72= b[72].drop(b[72].columns[[0,1,2]],axis=1)
r73 = b[73].drop(b[73].columns[[0,1,2]],axis=1)
r74 = b[74].drop(b[74].columns[[0,1,2]],axis=1)
r75= b[75].drop(b[75].columns[[0,1,2]],axis=1)
r76 = b[76].drop(b[76].columns[[0,1,2]],axis=1)
r77 = b[77].drop(b[77].columns[[0,1,2]],axis=1)##
r78 = b[78].drop(b[78].columns[[0,1,2]],axis=1)
r79 = b[79].drop(b[79].columns[[0,1,2]],axis=1)
r80 = b[80].drop(b[80].columns[[0,1,2]],axis=1)
r81 = b[81].drop(b[81].columns[[0,1,2]],axis=1)
r82 = b[82].drop(b[82].columns[[0,1,2]],axis=1)
r83 = b[83].drop(b[83].columns[[0,1,2]],axis=1)
r84 = b[84].drop(b[84].columns[[0,1,2]],axis=1)
r85 = b[85].drop(b[85].columns[[0,1,2]],axis=1)
r86 = b[86].drop(b[86].columns[[0,1,2]],axis=1)
r87 = b[87].drop(b[87].columns[[0,1,2]],axis=1)
r88 = b[88].drop(b[88].columns[[0,1,2]],axis=1)
r89 = b[89].drop(b[89].columns[[0,1,2]],axis=1)
r90 = b[90].drop(b[90].columns[[0,1,2]],axis=1)
r91 = b[91].drop(b[91].columns[[0,1,2]],axis=1)
r92 = b[92].drop(b[92].columns[[0,1,2]],axis=1)
r93 = b[93].drop(b[93].columns[[0,1,2]],axis=1)
r94 = b[94].drop(b[94].columns[[0,1,2]],axis=1)
r95 = b[95].drop(b[95].columns[[0,1,2]],axis=1)
r96 = b[96].drop(b[96].columns[[0,1,2]],axis=1)
r97 = b[97].drop(b[97].columns[[0,1,2]],axis=1)
r98 = b[98].drop(b[98].columns[[0,1,2]],axis=1)
r99 = b[99].drop(b[99].columns[[0,1,2]],axis=1)
r100 = b[100].drop(b[100].columns[[0,1,2]],axis=1)
r101= b[101].drop(b[101].columns[[0,1,2]],axis=1)
r102= b[102].drop(b[102].columns[[0,1,2]],axis=1)
r103= b[103].drop(b[103].columns[[0,1,2]],axis=1)
r104= b[104].drop(b[104].columns[[0,1,2]],axis=1)
r105= b[105].drop(b[105].columns[[0,1,2]],axis=1)
r106= b[106].drop(b[106].columns[[0,1,2]],axis=1)
r107= b[107].drop(b[107].columns[[0,1,2]],axis=1)
r108= b[108].drop(b[108].columns[[0,1,2]],axis=1)
r109= b[109].drop(b[109].columns[[0,1,2]],axis=1)
r110= b[110].drop(b[110].columns[[0,1,2]],axis=1)
r111= b[111].drop(b[111].columns[[0,1,2]],axis=1)
r112= b[112].drop(b[112].columns[[0,1,2]],axis=1)
r113= b[113].drop(b[113].columns[[0,1,2]],axis=1)
r114 = b[114].drop(b[114].columns[[0,1,2]],axis=1)
r115 = b[115].drop(b[115].columns[[0,1,2]],axis=1)





bentonite = [q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29,
             q30,q31,q32,q33,q34,q35,q36,q37,q38,q39,q40,q41,q42,q43,q44,q45,q46,q47,q48,q49,q50,q51,q52,q53,q54,q55,q56,q57,
            q58,q59,q60,q61,q62,q63,q64,q65,q66,q67,q68,q69,q70,q71,q72,q73,q74,q75,q76,q77,q78,q79,q80,q81,q82,q83,q84,q85,
            q86,q87,q88,q89,q90,q91,q92,q93,q94,q95,q96,q97,q98,q99,q100,q101,q102,q103,q104,q105,q106,q107,q108,q109,q110,
            q111,q112,q113,q114,q115]


sand = [r0,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13,r14,r15,r16,r17,r18,r19,r20,r21,r22,r23,r24,r25,r26,r27,r28,r29,
             r30,r31,r32,r33,r34,r35,r36,r37,r38,r39,r40,r41,r42,r43,r44,r45,r46,r47,r48,r49,r50,r51,r52,r53,r54,r55,r56,r57,
            r58,r59,r60,r61,r62,r63,r64,r65,r66,r67,r68,r69,r70,r71,r72,r73,r74,r75,r76,r77,r78,r79,r80,r81,r82,r83,r84,r85,
            r86,r87,r88,r89,r90,r91,r92,r93,r94,r95,r96,r97,r98,r99,r100,r101,r102,r103,r104,r105,r106,r107,r108,r109,r110,
            r111,r112,r113,r114,r115]


d2= d1[['gcs_', 'compactability_', 'active_clay_', 'wet_tensile_strength_', 'moisture_', 'permeability','shear_strength_']]
d3 = d1[['loi_', 'inert_fines_', 'volatile_matter_']]

df_names = []

comp1={}
for j in bentonite:
    if len(j)<5:
        print(j['component_id'],':','Insufficient data to build a model')
        continue
    else:
        pass
        
        for i in j:
            if i!='component_id':
                X=j.drop([i,'component_id'],axis=1) 
                y = j[i]
                if i == 'loi_':
                    break 

                sc=StandardScaler()
                X=sc.fit_transform(X)
                X = pd.DataFrame(X)

                from sklearn.model_selection import KFold 
                from sklearn import metrics
                n_split = 5 
                kf=KFold(n_splits=n_split,shuffle=True,random_state=0) 

                for model,name in zip([boost_rf],['boosted_random_forest']):
                    rmse_test=[]
                    r2_test=[]
                    adj_r2_test=[]
                    rmse_train=[]
                    r2_train=[]
                    adj_r2_train=[]
                    
                    for train,test in kf.split(X,y):
                        X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                        y_train,y_test=y.iloc[train],y.iloc[test]
                        model.fit(X_train,y_train) #fitting the model
                        y_predict_test=model.predict(X_test) #predicting the test result
                        mse_test=round(metrics.mean_squared_error(y_test,y_predict_test),3) #calculating mse
                        rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                        rSquare_test = round(r2_score((y_test),(y_predict_test)),3) #calculating rsquared for test
                        adj_rsquare_test= round(1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for test
                        r2_test.append(rSquare_test) #appending the result to the empty list
                        adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list

                        y_predict_train=model.predict(X_train) #predicting the result for train
                        mse_train=round(metrics.mean_squared_error(y_train,y_predict_train),3) #calculating mse for train
                        rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                        rSquare_train = round(r2_score((y_train),(y_predict_train)),3) #calculating rsquared for train
                        adj_rsquare_train= round(1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for train
                        r2_train.append(rSquare_train) #appending the result to the empty list
                        adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                    exec(f"df_{name}_{i} = pd.DataFrame()")
                    exec(f"df_{name}_{i}['Name'] =[name]*n_split")
                    exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
                    exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
                    exec(f"df_{name}_{i}['r2_train'] =r2_train")
                    exec(f"df_{name}_{i}['r2_test'] =r2_test")
                    exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
                    exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
                    exec(f"df_{name}_{i}['output'] = [i]*n_split")
                    df_names.append(f"df_{name}_{i}")
            
            
    collection = pd.DataFrame() #collecting the results in dataframe

    for nn in df_names:
        exec(f"collection=pd.concat([collection,{nn}])")

        collection['new_name'] = collection['output']


    df4 = collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

    abc = list(d2.columns)

    new = pd.DataFrame(columns=df4.columns)
    for names in range(len(abc)):
        val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
        df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
        new=pd.concat([new,df5])           
    
    comp=j.component_id.unique()[0]
    comp1[comp]=new
    print('\n')
    print('\033[1m'+ comp+':')
    print('\n')
    new[['r2_train','r2_test','adj_r2_train','adj_r2_test']] = round(new[['r2_train','r2_test','adj_r2_train','adj_r2_test']]*100,2)
    new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
    print(new)
    print('\n')
    
df_names = []

comp1={}
for j in sand:
    if len(j)<5:
        print(j['component_id'],':','Insufficient data to build a model')
        continue
    else:
        pass
        
        for i in j:
            if i!='component_id':
                X=j.drop([i,'component_id'],axis=1) 
                y = j[i]
                if i == 'gcs_':
                    break 

                sc=StandardScaler()
                X=sc.fit_transform(X)
                X = pd.DataFrame(X)

                from sklearn.model_selection import KFold 
                from sklearn import metrics
                n_split = 5 
                kf=KFold(n_splits=n_split,shuffle=True,random_state=0) 

                for model,name in zip([boost_rf],['boosted_random_forest']):
                    rmse_test=[]
                    r2_test=[]
                    adj_r2_test=[]
                    rmse_train=[]
                    r2_train=[]
                    adj_r2_train=[]
                    
                    for train,test in kf.split(X,y):
                        X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                        y_train,y_test=y.iloc[train],y.iloc[test]
                        model.fit(X_train,y_train) #fitting the model
                        y_predict_test=model.predict(X_test) #predicting the test result
                        mse_test=round(metrics.mean_squared_error(y_test,y_predict_test),3) #calculating mse
                        rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                        rSquare_test = round(r2_score((y_test),(y_predict_test)),3) #calculating rsquared for test
                        adj_rsquare_test= round(1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for test
                        r2_test.append(rSquare_test) #appending the result to the empty list
                        adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list

                        y_predict_train=model.predict(X_train) #predicting the result for train
                        mse_train=round(metrics.mean_squared_error(y_train,y_predict_train),3) #calculating mse for train
                        rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                        rSquare_train = round(r2_score((y_train),(y_predict_train)),3) #calculating rsquared for train
                        adj_rsquare_train= round(1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for train
                        r2_train.append(rSquare_train) #appending the result to the empty list
                        adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                    exec(f"df_{name}_{i} = pd.DataFrame()")
                    exec(f"df_{name}_{i}['Name'] =[name]*n_split")
                    exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
                    exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
                    exec(f"df_{name}_{i}['r2_train'] =r2_train")
                    exec(f"df_{name}_{i}['r2_test'] =r2_test")
                    exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
                    exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
                    exec(f"df_{name}_{i}['output'] = [i]*n_split")
                    df_names.append(f"df_{name}_{i}")
            
    
            
    collection = pd.DataFrame() #collecting the results in dataframe

    for nn in df_names:
        exec(f"collection=pd.concat([collection,{nn}])")

        collection['new_name'] = collection['output']


    df4 = collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

    abc = list(d3.columns)

    new = pd.DataFrame(columns=df4.columns)
    for names in range(len(abc)):
        val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
        df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
        new=pd.concat([new,df5])           
    
    comp=j.component_id.unique()[0]
    comp1[comp]=new
    print('\n')
    print('\033[1m'+ comp+':')
    print('\n')
    new[['r2_train','r2_test','adj_r2_train','adj_r2_test']] = round(new[['r2_train','r2_test','adj_r2_train','adj_r2_test']]*100,2)
    new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
    print(new)
    print('\n')


All
                       rmse_train  rmse_test  r_squared_train  r_squared_test  \
gcs_                         5.95      23.81            98.12           69.80   
compactability_              0.20       0.75            98.75           81.81   
active_clay_                 0.01       0.03            99.01           76.33   
wet_tensile_strength_        0.09       0.35            98.64           79.42   
moisture_                    0.01       0.03            98.55           89.36   
permeability                 1.12       4.02            97.65           69.76   
shear_strength_              4.07      15.45            97.00           56.67   
loi_                         0.01       0.03            99.13           89.88   
inert_fines_                 0.00       0.02            99.78           90.03   
volatile_matter_             0.01       0.04            99.27           83.53   

                       adj_r2_train  adj_r2_test  
gcs_                          98.11        69.69  
co

C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Group 1:
                       rmse_train  rmse_test  r_squared_train  r_squared_test  \
gcs_                         5.12      20.11            97.05           50.25   
compactability_              0.19       0.77            97.73           59.69   
active_clay_                 0.01       0.03            97.39           56.68   
wet_tensile_strength_        0.11       0.38            97.95           73.67   
moisture_                    0.01       0.03            99.03           84.96   
permeability                 1.28       4.28            97.06           65.90   
shear_strength_              4.02      16.41            96.17           35.01   
loi_                         0.01       0.03            99.05           84.95   
inert_fines_                 0.01       0.02            98.93           84.01   
volatile_matter_             0.01       0.04            98.90           80.70   

                       adj_r2_train  adj_r2_test  
gcs_                          96.84        46.73

C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Group 2:
                       rmse_train  rmse_test  r_squared_train  r_squared_test  \
gcs_                         7.15      28.40            97.59           60.86   
compactability_              0.19       0.76            98.63           77.66   
active_clay_                 0.01       0.03            98.39           72.66   
wet_tensile_strength_        0.10       0.38            98.45           76.04   
moisture_                    0.01       0.03            99.20           86.73   
permeability                 1.00       3.64            98.06           73.97   
shear_strength_              4.56      16.47            95.59           41.68   
loi_                         0.01       0.02            99.57           90.83   
inert_fines_                 0.00       0.02            99.23           80.91   
volatile_matter_             0.01       0.04            99.17           82.67   

                       adj_r2_train  adj_r2_test  
gcs_                          97.53        59.99

...



5361:


                  adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  \
loi_                      NaN        157.76      NaN    -30.00       0.13   
inert_fines_              NaN        140.22      NaN      9.48       0.01   
volatile_matter_          NaN        161.24      NaN    -37.78       0.26   

                  rmse_train  
loi_                    0.07  
inert_fines_            0.00  
volatile_matter_        0.15  


3066    5259/60
4077    5259/60
4423    5259/60
Name: component_id, dtype: object : Insufficient data to build a model
3176    5362
Name: component_id, dtype: object : Insufficient data to build a model
3182    RPT 82
Name: component_id, dtype: object : Insufficient data to build a model
3327    6811
Name: component_id, dtype: object : Insufficient data to build a model
3457    7368
Name: component_id, dtype: object : Insufficient data to build a model
3506    7427-7
Name: component_id, dtype: object : Insufficient data to build a model
3643    7284

C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\r



7328:


                  adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  \
loi_                    162.5         162.5      0.0       0.0        0.0   
inert_fines_            162.5         162.5      0.0       0.0        0.0   
volatile_matter_        162.5         162.5      0.0       0.0        0.0   

                  rmse_train  
loi_                     0.0  
inert_fines_             0.0  
volatile_matter_         0.0  




C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\r



7329:


                  adj_r2_test  adj_r2_train  r2_test  r2_train  rmse_test  \
loi_                    161.5        146.30      1.6     25.90       0.09   
inert_fines_            225.0        157.70   -100.0      7.66       0.02   
volatile_matter_        225.0        150.44   -100.0     19.30       0.10   

                  rmse_train  
loi_                    0.06  
inert_fines_            0.04  
volatile_matter_        0.08  




# MAXWELL:

In [ ]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

#loading the prepared sand data
dfp = pd.read_excel('Preparedsand_01-Jan-2019_TO_15-Mar-2020_Disa.xlsx',skiprows=[0,1,2,3,4])
dfp.drop(dfp.columns[[3, 14, 27]], axis = 1, inplace = True)#removing heat number, specimen weight and relative humidity
dfp = dfp.dropna(axis=1,how='all') #removing the columns with full null values
dfp.fillna(dfp.median(), inplace=True)  #replacing rest of the null values with median values
dfb = pd.read_excel('Bentonite_01-Jan-2019_TO_15-Mar-2020_Disa.xlsx',skiprows=[0,1,2,3,4])
dfb.drop(dfb.columns[[1, 2, 3, 4]], axis = 1, inplace = True)#removing batch no, supplier, quantity, and grade columns
dfb.columns = dfb.columns.str.replace(' ','_').str.replace('(','').str.replace(')','').str.replace('%','_raw').str.replace('kg/cm2','_raw').str.replace('gm/cm2','_raw')
dfb = dfb.dropna(axis=1,how='all') #removing the columns with full null values
dff_b = pd.merge(dfp,dfb,on='Date',how='left')
dff_b.fillna(method='ffill',inplace=True)
dff_b.fillna(method='bfill',inplace=True)
cols = list(dff_b)
cols[0],cols[1],cols[2],cols[3],cols[4],cols[5],cols[6],cols[7],cols[8],cols[9],cols[10],cols[11],cols[12],cols[13],cols[14],cols[15],cols[16],cols[17] = cols[0],cols[1],cols[2],cols[3],cols[4],cols[5],cols[6],cols[7],cols[9],cols[13],cols[8],cols[10],cols[11],cols[12],cols[14],cols[15],cols[16],cols[17]
dff_b = dff_b.loc[:,cols]
df1 = dff_b.copy()
db = dff_b.copy()
df1.drop(df1.columns[[0,1,2,3]],axis=1,inplace=True)
dfs = pd.read_excel('Freshsilicasand_01-Jan-2019_TO_15-Mar-2020_Disa.xlsx',skiprows=[0,1,2,3,4]) #loading FSS raw material data
dfs.drop(dfs.columns[[1,2,3,5,8]], axis = 1, inplace = True)#removing company, origin, application and sample weight columns.
dfs.columns = dfs.columns.str.replace(' ','_').str.replace('(','').str.replace(')','').str.replace('%','_raw').str.replace('kg/cm2','_raw').str.replace('gm/cm2','_raw').str.replace('_no','_raw')
dfs = dfs.dropna(axis=1,how='all') #removing the columns with full null values
dff_fs = pd.merge(dfp,dfs,on='Date',how='left')
dff_fs.fillna(method='ffill',inplace=True)
dff_fs.fillna(method='bfill',inplace=True)
cols = list(dff_fs)
cols[0],cols[1],cols[2],cols[3],cols[4],cols[5],cols[6],cols[7],cols[8],cols[9],cols[10],cols[11],cols[12],cols[13],cols[14],cols[15],cols[16],cols[17] =cols[0],cols[1],cols[2],cols[3],cols[14],cols[4],cols[5],cols[6],cols[7],cols[8],cols[9],cols[10],cols[11],cols[12],cols[13],cols[15],cols[16],cols[17]
dff_fs = dff_fs.loc[:,cols]
dfs = dff_fs.copy()
df3 = dff_fs.copy()
df3.drop(df3.columns[[0,1,2,3]],axis=1,inplace=True)
cols = list(dfp)
cols[0],cols[1],cols[2],cols[3],cols[4],cols[5],cols[6],cols[7],cols[8],cols[9],cols[10],cols[11],cols[12],cols[13],cols[14],cols[15] =cols[0],cols[1],cols[2],cols[3],cols[8],cols[11],cols[12],cols[4],cols[5],cols[6],cols[7],cols[9],cols[10],cols[13],cols[14],cols[15]
dfp = dfp.loc[:,cols]
df2 = dfp.copy()
dpr = dfp.copy()
df2.drop(df2.columns[[0,1,2,3]],axis=1,inplace=True)
#df1 has prepared sand and bentonite data
#df2 has only the prepared sand data
#df3 has prepared sand and FSS data
d1 = df1.copy()
d1.drop(d1.columns[[7, 11,12,13,14,15]], axis = 1, inplace = True)
rf = RandomForestRegressor(n_estimators=100,random_state=0)
boost_rf = AdaBoostRegressor(base_estimator = rf, n_estimators=50,random_state=0)
df1.columns = df1.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
df2.columns = df2.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
df3.columns = df3.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
d1.columns = d1.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in df1:
    X=df1.drop(i,axis=1) #removing the target variable
    y = df1[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in df2:
    X=df2.drop(i,axis=1)
    y = df2[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
#similar to df1 and df2 , all the same steps are carried out for df3
for i in df3:
    X=df3.drop(i,axis=1)
    y = df3[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'All')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)
dff_b.columns = dff_b.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
dff_fs.columns = dff_fs.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
dfp.columns = dfp.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')

#groupwise:

group_high_b= dff_b.loc[dff_b['component_id'].isin(['AVA001','KPJ000','KPJ001','KPJ002','KPJ003','MCW010',
                                                   'MCW011','MCW021','PCF001','PCF002','RAB006','RAB007','RAB009',
                                                   'RAB010','RAE002','RAE004','RAE020','RAE028','RAE029','RAE030',
                                                   'RAE032','SG001','SG002','SG003','SGC001','SGC004','USH002',
                                                   'USH005','USH006','USH007','USH008','USH009'])]


group_high_ps = dfp.loc[dfp['component_id'].isin(['AVA001','KPJ000','KPJ001','KPJ002','KPJ003','MCW010',
                                                   'MCW011','MCW021','PCF001','PCF002','RAB006','RAB007','RAB009',
                                                   'RAB010','RAE002','RAE004','RAE020','RAE028','RAE029','RAE030',
                                                   'RAE032','SG001','SG002','SG003','SGC001','SGC004','USH002',
                                                   'USH005','USH006','USH007','USH008','USH009'])]
group_high_fs = dff_fs.loc[dff_fs['component_id'].isin(['AVA001','KPJ000','KPJ001','KPJ002','KPJ003','MCW010',
                                                   'MCW011','MCW021','PCF001','PCF002','RAB006','RAB007','RAB009',
                                                   'RAB010','RAE002','RAE004','RAE020','RAE028','RAE029','RAE030',
                                                   'RAE032','SG001','SG002','SG003','SGC001','SGC004','USH002',
                                                   'USH005','USH006','USH007','USH008','USH009'])]
group_high_b.drop(group_high_b.columns[[0,1,2,3]],axis=1,inplace=True)
group_high_fs.drop(group_high_fs.columns[[0,1,2,3]],axis=1,inplace=True)
group_high_ps.drop(group_high_ps.columns[[0,1,2,3]],axis=1,inplace=True)

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group_high_b:
    X=group_high_b.drop(i,axis=1) #removing the target variable
    y = group_high_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group_high_ps:
    X=group_high_ps.drop(i,axis=1)
    y = group_high_ps[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
#similar to df1 and df2 , all the same steps are carried out for df3
for i in group_high_fs:
    X=group_high_fs.drop(i,axis=1)
    y = group_high_fs[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group High:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

group_medium_b= dff_b.loc[dff_b['component_id'].isin(['BHR002','CRH002','CRH010','GAC005','GAC008','GAC009','GAC013',
                                                     'HFB51 HOUSING','ISP006','ISP009','ISP010','ISP011','KAR005',
                                                     'KAR011','KAR012','MAR003','MAR004','MAR005','MAR006','MAR007',
                                                     'MAR011','MAR012','MAR016','MAR020','MAR021','MAR023',
                                                     'MAR024','MAR026','MAR028','MAR029','MAR030','MAR031',
                                                     'MAR034','MAR035','MAR039','MAR040','MAR042','MAR050','MCW006',
                                                      'MCW007','MCW012','MCW015','MCW016','MCW017','MCW018','MCW019',
                                                      'MCW020','MCW022','NUM003','NUM008','PCF003','RAB001','RAB005',
                                                      'RAE003','RAE010','RAE013','RAE015','RAE017','RAE025','RAE035',
                                                      'RAM003','RAM005','RAM006','RAM009','RAM010','SUB001','USH001',
                                                      'USH003','USH004','VEL001','VEL002'])]

group_medium_ps = dfp.loc[dfp['component_id'].isin(['BHR002','CRH002','CRH010','GAC005','GAC008','GAC009','GAC013',
                                                     'HFB51 HOUSING','ISP006','ISP009','ISP010','ISP011','KAR005',
                                                     'KAR011','KAR012','MAR003','MAR004','MAR005','MAR006','MAR007',
                                                     'MAR011','MAR012','MAR016','MAR020','MAR021','MAR023',
                                                     'MAR024','MAR026','MAR028','MAR029','MAR030','MAR031',
                                                     'MAR034','MAR035','MAR039','MAR040','MAR042','MAR050','MCW006',
                                                      'MCW007','MCW012','MCW015','MCW016','MCW017','MCW018','MCW019',
                                                      'MCW020','MCW022','NUM003','NUM008','PCF003','RAB001','RAB005',
                                                      'RAE003','RAE010','RAE013','RAE015','RAE017','RAE025','RAE035',
                                                      'RAM003','RAM005','RAM006','RAM009','RAM010','SUB001','USH001',
                                                      'USH003','USH004','VEL001','VEL002'])]

group_medium_fs = dff_fs.loc[dff_fs['component_id'].isin(['BHR002','CRH002','CRH010','GAC005','GAC008','GAC009','GAC013',
                                                     'HFB51 HOUSING','ISP006','ISP009','ISP010','ISP011','KAR005',
                                                     'KAR011','KAR012','MAR003','MAR004','MAR005','MAR006','MAR007',
                                                     'MAR011','MAR012','MAR016','MAR020','MAR021','MAR023',
                                                     'MAR024','MAR026','MAR028','MAR029','MAR030','MAR031',
                                                     'MAR034','MAR035','MAR039','MAR040','MAR042','MAR050','MCW006',
                                                      'MCW007','MCW012','MCW015','MCW016','MCW017','MCW018','MCW019',
                                                      'MCW020','MCW022','NUM003','NUM008','PCF003','RAB001','RAB005',
                                                      'RAE003','RAE010','RAE013','RAE015','RAE017','RAE025','RAE035',
                                                      'RAM003','RAM005','RAM006','RAM009','RAM010','SUB001','USH001',
                                                      'USH003','USH004','VEL001','VEL002'])]


group_medium_b.drop(group_medium_b.columns[[0,1,2,3]],axis=1,inplace=True)
group_medium_fs.drop(group_medium_fs.columns[[0,1,2,3]],axis=1,inplace=True)
group_medium_ps.drop(group_medium_ps.columns[[0,1,2,3]],axis=1,inplace=True)

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group_medium_b:
    X=group_medium_b.drop(i,axis=1) #removing the target variable
    y = group_medium_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group_medium_ps:
    X=group_medium_ps.drop(i,axis=1)
    y = group_medium_ps[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
#similar to df1 and df2 , all the same steps are carried out for df3
for i in group_medium_fs:
    X=group_medium_fs.drop(i,axis=1)
    y = group_medium_fs[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group Medium:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)


group_low_b= dff_b.loc[dff_b['component_id'].isin(['AEW000','AEW001','AEW002','BER001','BER002','BER003',
                                                  'BHR001','BTP001','BTP002','BTP003','CRH001','CRH003',
                                                  'CRH004','CRH005','CRH006','CRH007','CRH008','CRH009',
                                                  'CRH011','CRH012','CRH013','CRH014','CRH015','CRH016',
                                                  'CRH017','CRH018','DEE001','DEE002','DEE003','DEV001',
                                                  'DYN001','DYN002','DYN003','DYN004','DYN005','DYN006','DYN007',
                                                  'DYN008','DYN009','G&C012','G&C013','GAC001','GAC002','GAC003',
                                                  'GAC004','GAC006','GAC007','GAC010','GAC011','GAC012','GRATE - 6008/94473',
                                                  'ISP001','ISP002','ISP003','ISP004','ISP005','ISP008','ISP012',
                                                  'ISP013','ISP014','KAR002','KAR003','KAR004','KAR006','KAR007','KAR009',
                                                  'KAR010','KAR013','KAR014','MAH001','MAH002','MAR001','MAR002',
                                                  'MAR008','MAR010','MAR013','MAR014','MAR015','MAR017','MAR018','MAR019',
                                                  'MAR022','MAR025','MAR027','MAR032','MAR033','MAR036','MAR037','MAR038',
                                                   'MAR041','MAR043','MAR044','MAR045','MAR046','MAR047','MAR048',
                                                   'MAR049','MAR051','MAR052','MAR053','MCW001','MCW003','MCW004',
                                                   'MCW005','MCW013','MCW014','MPP001','MPP002','NUM001','NUM002',
                                                   'NUM004','NUM005','NUM006','NUM007','NUM009','PCF004','PR001',
                                                  'PR002','PR003','PSM001','RAB002','RAB003','RAB004','RAB008','RAB011',
                                                   'RAB012','RAE001','RAE005','RAE006','RAE007','RAE008','RAE009',
                                                   'RAE011','RAE012','RAE014','RAE016','RAE018','RAE019','RAE021',
                                                   'RAE024','RAE026','RAE027','RAE034','RAM001','RAM002','RAM004','RAM008',
                                                   'RAN001','REX001','REX002','RYD002','RYD003','SEA001','SEA002','SEA003',
                                                   'SGC002','SGC003','SIG001','SPC001','SPC002','UNI001','UNI002','UNI003',
                                                   'UNI004','UNI005','UNI006','VEL000','VIB001','VIB002','VIB003',
                                                   'VIB004','VIB005','VIB006','VIB007','VIB008'])]

group_low_ps = dfp.loc[dfp['component_id'].isin(['AEW000','AEW001','AEW002','BER001','BER002','BER003',
                                                  'BHR001','BTP001','BTP002','BTP003','CRH001','CRH003',
                                                  'CRH004','CRH005','CRH006','CRH007','CRH008','CRH009',
                                                  'CRH011','CRH012','CRH013','CRH014','CRH015','CRH016',
                                                  'CRH017','CRH018','DEE001','DEE002','DEE003','DEV001',
                                                  'DYN001','DYN002','DYN003','DYN004','DYN005','DYN006','DYN007',
                                                  'DYN008','DYN009','G&C012','G&C013','GAC001','GAC002','GAC003',
                                                  'GAC004','GAC006','GAC007','GAC010','GAC011','GAC012','GRATE - 6008/94473',
                                                  'ISP001','ISP002','ISP003','ISP004','ISP005','ISP008','ISP012',
                                                  'ISP013','ISP014','KAR002','KAR003','KAR004','KAR006','KAR007','KAR009',
                                                  'KAR010','KAR013','KAR014','MAH001','MAH002','MAR001','MAR002',
                                                  'MAR008','MAR010','MAR013','MAR014','MAR015','MAR017','MAR018','MAR019',
                                                  'MAR022','MAR025','MAR027','MAR032','MAR033','MAR036','MAR037','MAR038',
                                                   'MAR041','MAR043','MAR044','MAR045','MAR046','MAR047','MAR048',
                                                   'MAR049','MAR051','MAR052','MAR053','MCW001','MCW003','MCW004',
                                                   'MCW005','MCW013','MCW014','MPP001','MPP002','NUM001','NUM002',
                                                   'NUM004','NUM005','NUM006','NUM007','NUM009','PCF004','PR001',
                                                  'PR002','PR003','PSM001','RAB002','RAB003','RAB004','RAB008','RAB011',
                                                   'RAB012','RAE001','RAE005','RAE006','RAE007','RAE008','RAE009',
                                                   'RAE011','RAE012','RAE014','RAE016','RAE018','RAE019','RAE021',
                                                   'RAE024','RAE026','RAE027','RAE034','RAM001','RAM002','RAM004','RAM008',
                                                   'RAN001','REX001','REX002','RYD002','RYD003','SEA001','SEA002','SEA003',
                                                   'SGC002','SGC003','SIG001','SPC001','SPC002','UNI001','UNI002','UNI003',
                                                   'UNI004','UNI005','UNI006','VEL000','VIB001','VIB002','VIB003',
                                                   'VIB004','VIB005','VIB006','VIB007','VIB008'])]

group_low_fs = dff_fs.loc[dff_fs['component_id'].isin(['AEW000','AEW001','AEW002','BER001','BER002','BER003',
                                                  'BHR001','BTP001','BTP002','BTP003','CRH001','CRH003',
                                                  'CRH004','CRH005','CRH006','CRH007','CRH008','CRH009',
                                                  'CRH011','CRH012','CRH013','CRH014','CRH015','CRH016',
                                                  'CRH017','CRH018','DEE001','DEE002','DEE003','DEV001',
                                                  'DYN001','DYN002','DYN003','DYN004','DYN005','DYN006','DYN007',
                                                  'DYN008','DYN009','G&C012','G&C013','GAC001','GAC002','GAC003',
                                                  'GAC004','GAC006','GAC007','GAC010','GAC011','GAC012','GRATE - 6008/94473',
                                                  'ISP001','ISP002','ISP003','ISP004','ISP005','ISP008','ISP012',
                                                  'ISP013','ISP014','KAR002','KAR003','KAR004','KAR006','KAR007','KAR009',
                                                  'KAR010','KAR013','KAR014','MAH001','MAH002','MAR001','MAR002',
                                                  'MAR008','MAR010','MAR013','MAR014','MAR015','MAR017','MAR018','MAR019',
                                                  'MAR022','MAR025','MAR027','MAR032','MAR033','MAR036','MAR037','MAR038',
                                                   'MAR041','MAR043','MAR044','MAR045','MAR046','MAR047','MAR048',
                                                   'MAR049','MAR051','MAR052','MAR053','MCW001','MCW003','MCW004',
                                                   'MCW005','MCW013','MCW014','MPP001','MPP002','NUM001','NUM002',
                                                   'NUM004','NUM005','NUM006','NUM007','NUM009','PCF004','PR001',
                                                  'PR002','PR003','PSM001','RAB002','RAB003','RAB004','RAB008','RAB011',
                                                   'RAB012','RAE001','RAE005','RAE006','RAE007','RAE008','RAE009',
                                                   'RAE011','RAE012','RAE014','RAE016','RAE018','RAE019','RAE021',
                                                   'RAE024','RAE026','RAE027','RAE034','RAM001','RAM002','RAM004','RAM008',
                                                   'RAN001','REX001','REX002','RYD002','RYD003','SEA001','SEA002','SEA003',
                                                   'SGC002','SGC003','SIG001','SPC001','SPC002','UNI001','UNI002','UNI003',
                                                   'UNI004','UNI005','UNI006','VEL000','VIB001','VIB002','VIB003',
                                                   'VIB004','VIB005','VIB006','VIB007','VIB008'])]

group_low_b.drop(group_low_b.columns[[0,1,2,3]],axis=1,inplace=True)
group_low_fs.drop(group_low_fs.columns[[0,1,2,3]],axis=1,inplace=True)
group_low_ps.drop(group_low_ps.columns[[0,1,2,3]],axis=1,inplace=True)

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group_low_b:
    X=group_low_b.drop(i,axis=1) #removing the target variable
    y = group_low_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group_low_ps:
    X=group_low_ps.drop(i,axis=1)
    y = group_low_ps[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
#similar to df1 and df2 , all the same steps are carried out for df3
for i in group_low_fs:
    X=group_low_fs.drop(i,axis=1)
    y = group_low_fs[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group Low:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

group_rba_b= dff_b.loc[dff_b['component_id'].isin(['MAR054','MAR055','MAR056','RBA00','RBA001','RBA002','RBA003',
                                                   'RBA004','RBA005','RBA006','RBA007','RBA008','RBA009','RBA010',
                                                   'RBA011','RBA013','RBA014'])]

group_rba_ps = dfp.loc[dfp['component_id'].isin(['MAR054','MAR055','MAR056','RBA00','RBA001','RBA002','RBA003',
                                                   'RBA004','RBA005','RBA006','RBA007','RBA008','RBA009','RBA010',
                                                   'RBA011','RBA013','RBA014'])]

group_rba_fs = dff_fs.loc[dff_fs['component_id'].isin(['MAR054','MAR055','MAR056','RBA00','RBA001','RBA002','RBA003',
                                                   'RBA004','RBA005','RBA006','RBA007','RBA008','RBA009','RBA010',
                                                   'RBA011','RBA013','RBA014'])]

group_rba_b.drop(group_rba_b.columns[[0,1,2,3]],axis=1,inplace=True)
group_rba_fs.drop(group_rba_fs.columns[[0,1,2,3]],axis=1,inplace=True)
group_rba_ps.drop(group_rba_ps.columns[[0,1,2,3]],axis=1,inplace=True)

df_names = [] #empty list to append all the results so that we can have all the results in a dataframe

for i in group_rba_b:
    X=group_rba_b.drop(i,axis=1) #removing the target variable
    y = group_rba_b[i] # having only the target variable
    
    if i == 'loi_':
        break #breaking the loop because from loi we need to use df2 since loi deals with only prepared sand data without
              #bentonite

    sc=StandardScaler()
    X=sc.fit_transform(X) #scaling the X variable.
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0) #kfold cross validation method is used to split the data

    for model,name in zip([boost_rf],['boosted_random_forest']): #defining the required models.
            #empty lists are used to append the results after the calculation.
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the test result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared for test
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for test
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result for train
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse for train
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared for train
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2 for train
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
            #exec is used so that we can collect all the results in a form of dataframe at the end.    
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
            
            
#similar to df1 , all the same steps are carried out for df2           
for i in group_rba_ps:
    X=group_rba_ps.drop(i,axis=1)
    y = group_rba_ps[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
#similar to df1 and df2 , all the same steps are carried out for df3
for i in group_rba_fs:
    X=group_rba_fs.drop(i,axis=1)
    y = group_rba_fs[i]
    
    if i == 'gcs_':
        break


    sc=StandardScaler()
    X=sc.fit_transform(X)
    X = pd.DataFrame(X)

    from sklearn.model_selection import KFold #using kflod cross validating method for prediction
    from sklearn import metrics
    n_split = 5 #5times r2,adj_r2 and rmse will be calculated and the mean of it will be the result
    kf=KFold(n_splits=n_split,shuffle=True,random_state=0)

    for model,name in zip([boost_rf],['boosted_random_forest']):
            rmse_test=[]
            r_squared_test=[]
            adj_r2_test=[]
            rmse_train=[]
            r_squared_train=[]
            adj_r2_train=[]
            for train,test in kf.split(X,y):
                X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                y_train,y_test=y.iloc[train],y.iloc[test]
                model.fit(X_train,y_train) #fitting the model
                y_predict_test=model.predict(X_test) #predicting the result
                mse_test=metrics.mean_squared_error(y_test,y_predict_test) #calculating mse
                rmse_test.append(np.sqrt(mse_test)) # calculating rmse
                rSquare_test = r2_score((y_test),(y_predict_test)) #calculating rsquared
                adj_rsquare_test= 1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_test.append(rSquare_test) #appending the result to the empty list
                adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list
                
                y_predict_train=model.predict(X_train) #predicting the result
                mse_train=metrics.mean_squared_error(y_train,y_predict_train) #calculating mse
                rmse_train.append(np.sqrt(mse_train)) # calculating rmse
                rSquare_train = r2_score((y_train),(y_predict_train)) #calculating rsquared
                adj_rsquare_train= 1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1) #calculating adjusted r2
                r_squared_train.append(rSquare_train) #appending the result to the empty list
                adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                
            exec(f"df_{name}_{i} = pd.DataFrame()")
            exec(f"df_{name}_{i}['Name'] =[name]*n_split")
            exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
            exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
            exec(f"df_{name}_{i}['r_squared_train'] =r_squared_train")
            exec(f"df_{name}_{i}['r_squared_test'] =r_squared_test")
            exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
            exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
            exec(f"df_{name}_{i}['output'] = [i]*n_split")
            df_names.append(f"df_{name}_{i}")
            
collection = pd.DataFrame() #collecting the results in dataframe

for nn in df_names:
    
    exec(f"collection=pd.concat([collection,{nn}])")

collection['new_name'] = collection['output']


df4= collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

abc = list(d1.columns)

new = pd.DataFrame(columns=df4.columns)
for names in range(len(abc)):
   
    val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
    df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
    new=pd.concat([new,df5])
    
print('\033[1m' + 'Group RBA:')
new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']] = round(new[['r_squared_train','r_squared_test','adj_r2_train','adj_r2_test']]*100,2)
new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
print(new)

#For each component ID:
db.columns = db.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
dfs.columns = dfs.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')
dpr.columns = dpr.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%','').str.replace('/','').str.replace('kgcm2','').str.replace('gmcm2','').str.replace('_no','')

#prepared sand and bentonite
a=[]
for each in db.component_id.unique():
    a.append(db.loc[db.component_id==each,:])
#only prepared sand
b=[]
for each in dpr.component_id.unique():
    b.append(dpr.loc[dpr.component_id==each,:])
    
#prepared sand and fss
c=[]
for each in dfs.component_id.unique():
    c.append(dfs.loc[dfs.component_id==each,:])
    
data_bentonite = a[0],a[1],a[2],a[3],a[4],a[5],a[6],a[7],a[8],a[9],a[10],a[11],a[12],a[13],a[14],a[15],a[16],a[17],a[18],a[19],a[20],a[21],a[22],a[23],a[24],a[25],a[26],a[27],a[28],a[29],a[30],a[31],a[32],a[33],a[34],a[35],a[36],a[37],a[38],a[39],a[40],a[41],a[42],a[43],a[44],a[45],a[46],a[47],a[48],a[49],a[50],a[51],a[52],a[53],a[54],a[55],a[56],a[57],a[58],a[59],a[60],a[61],a[62],a[63],a[64],a[65],a[66],a[67],a[68],a[69],a[70],a[71],a[72],a[73],a[74],a[75],a[76],a[77],a[78],a[79],a[80],a[81],a[82],a[83],a[84],a[85],a[86],a[87],a[88],a[89],a[90],a[91],a[92],a[93],a[94],a[95],a[96],a[97],a[98],a[99],a[100],a[101],a[102],a[103],a[104],a[105],a[106],a[107],a[108],a[109],a[110],a[111],a[112],a[113],a[114],a[115],a[116],a[117],a[118],a[119],a[120],a[121],a[122],a[123],a[124],a[125],a[126],a[127],a[128],a[129],a[130],a[131],a[132],a[133],a[134],a[135],a[136],a[137],a[138],a[139],a[140],a[141],a[142],a[143],a[144],a[145],a[146],a[147],a[148],a[149],a[150],a[151],a[152],a[153],a[154],a[155],a[156],a[157],a[158],a[159],a[160],a[161],a[162],a[163],a[164],a[165],a[166],a[167],a[168],a[169],a[170],a[171]
data_sand = b[0],b[1],b[2],b[3],b[4],b[5],b[6],b[7],b[8],b[9],b[10],b[11],b[12],b[13],b[14],b[15],b[16],b[17],b[18],b[19],b[20],b[21],b[22],b[23],b[24],b[25],b[26],b[27],b[28],b[29],b[30],b[31],b[32],b[33],b[34],b[35],b[36],b[37],b[38],b[39],b[40],b[41],b[42],b[43],b[44],b[45],b[46],b[47],b[48],b[49],b[50],b[51],b[52],b[53],b[54],b[55],b[56],b[57],b[58],b[59],b[60],b[61],b[62],b[63],b[64],b[65],b[66],b[67],b[68],b[69],b[70],b[71],b[72],b[73],b[74],b[75],b[76],b[77],b[78],b[79],b[80],b[81],b[82],b[83],b[84],b[85],b[86],b[87],b[88],b[89],b[90],b[91],b[92],b[93],b[94],b[95],b[96],b[97],b[98],b[99],b[100],b[101],b[102],b[103],b[104],b[105],b[106],b[107],b[108],b[109],b[110],b[111],b[112],b[113],b[114],b[115],b[116],b[117],b[118],b[119],b[120],b[121],b[122],b[123],b[124],b[125],b[126],b[127],b[128],b[129],b[130],b[131],b[132],b[133],b[134],b[135],b[136],b[137],b[138],b[139],b[140],b[141],b[142],b[143],b[144],b[145],b[146],b[147],b[148],b[149],b[150],b[151],b[152],b[153],b[154],b[155],b[156],b[157],b[158],b[159],b[160],b[161],b[162],b[163],b[164],b[165],b[166],b[167],b[168],b[169],b[170],b[171]
data_fss = c[0],c[1],c[2],c[3],c[4],c[5],c[6],c[7],c[8],c[9],c[10],c[11],c[12],c[13],c[14],c[15],c[16],c[17],c[18],c[19],c[20],c[21],c[22],c[23],c[24],c[25],c[26],c[27],c[28],c[29],c[30],c[31],c[32],c[33],c[34],c[35],c[36],c[37],c[38],c[39],c[40],c[41],c[42],c[43],c[44],c[45],c[46],c[47],c[48],c[49],c[50],c[51],c[52],c[53],c[54],c[55],c[56],c[57],c[58],c[59],c[60],c[61],c[62],c[63],c[64],c[65],c[66],c[67],c[68],c[69],c[70],c[71],c[72],c[73],c[74],c[75],c[76],c[77],c[78],c[79],c[80],c[81],c[82],c[83],c[84],c[85],c[86],c[87],c[88],c[89],c[90],c[91],c[92],c[93],c[94],c[95],c[96],c[97],c[98],c[99],c[100],c[101],c[102],c[103],c[104],c[105],c[106],c[107],c[108],c[109],c[110],c[111],c[112],c[113],c[114],c[115],c[116],c[117],c[118],c[119],c[120],c[121],c[122],c[123],c[124],c[125],c[126],c[127],c[128],c[129],c[130],c[131],c[132],c[133],c[134],c[135],c[136],c[137],c[138],c[139],c[140],c[141],c[142],c[143],c[144],c[145],c[146],c[147],c[148],c[149],c[150],c[151],c[152],c[153],c[154],c[155],c[156],c[157],c[158],c[159],c[160],c[161],c[162],c[163],c[164],c[165],c[166],c[167],c[168],c[169],c[170],c[171]

#prepared sand and bentonite dataframes:
q0 = a[0].drop(a[0].columns[[0,1,2]],axis=1)
q1 = a[1].drop(a[1].columns[[0,1,2]],axis=1)
q2 = a[2].drop(a[2].columns[[0,1,2]],axis=1)
q3 = a[3].drop(a[3].columns[[0,1,2]],axis=1)
q4 = a[4].drop(a[4].columns[[0,1,2]],axis=1)
q5 = a[5].drop(a[5].columns[[0,1,2]],axis=1)
q6 = a[6].drop(a[6].columns[[0,1,2]],axis=1)
q7 = a[7].drop(a[7].columns[[0,1,2]],axis=1)
q8 = a[8].drop(a[8].columns[[0,1,2]],axis=1)
q9 = a[9].drop(a[9].columns[[0,1,2]],axis=1)
q10 = a[10].drop(a[10].columns[[0,1,2]],axis=1)
q11 = a[10].drop(a[11].columns[[0,1,2]],axis=1)
q12 = a[12].drop(a[12].columns[[0,1,2]],axis=1)
q13 = a[13].drop(a[13].columns[[0,1,2]],axis=1)
q14 = a[14].drop(a[14].columns[[0,1,2]],axis=1)
q15 = a[15].drop(a[15].columns[[0,1,2]],axis=1)
q16 = a[16].drop(a[16].columns[[0,1,2]],axis=1)
q17 = a[17].drop(a[17].columns[[0,1,2]],axis=1)
q18 = a[18].drop(a[18].columns[[0,1,2]],axis=1)
q19 = a[19].drop(a[19].columns[[0,1,2]],axis=1)
q20 = a[20].drop(a[20].columns[[0,1,2]],axis=1)
q21 = a[21].drop(a[21].columns[[0,1,2]],axis=1)
q22 = a[22].drop(a[22].columns[[0,1,2]],axis=1)
q23 = a[23].drop(a[23].columns[[0,1,2]],axis=1)
q24 = a[24].drop(a[24].columns[[0,1,2]],axis=1)
q25 = a[25].drop(a[25].columns[[0,1,2]],axis=1)
q26 = a[26].drop(a[26].columns[[0,1,2]],axis=1)
q27 = a[27].drop(a[27].columns[[0,1,2]],axis=1)
q28 = a[28].drop(a[28].columns[[0,1,2]],axis=1)
q29 = a[29].drop(a[29].columns[[0,1,2]],axis=1)
q30 = a[30].drop(a[30].columns[[0,1,2]],axis=1)
q31 = a[31].drop(a[31].columns[[0,1,2]],axis=1)
q32 = a[32].drop(a[32].columns[[0,1,2]],axis=1)
q33 = a[33].drop(a[33].columns[[0,1,2]],axis=1)
q34 = a[34].drop(a[34].columns[[0,1,2]],axis=1)
q35 = a[35].drop(a[35].columns[[0,1,2]],axis=1)
q36 = a[36].drop(a[36].columns[[0,1,2]],axis=1)
q37 = a[37].drop(a[37].columns[[0,1,2]],axis=1)
q38 = a[38].drop(a[38].columns[[0,1,2]],axis=1)
q39 = a[39].drop(a[39].columns[[0,1,2]],axis=1)
q40 = a[40].drop(a[40].columns[[0,1,2]],axis=1)
q41 = a[41].drop(a[41].columns[[0,1,2]],axis=1)
q42 = a[42].drop(a[42].columns[[0,1,2]],axis=1)
q43 = a[43].drop(a[43].columns[[0,1,2]],axis=1)
q44 = a[44].drop(a[44].columns[[0,1,2]],axis=1)
q45 = a[45].drop(a[45].columns[[0,1,2]],axis=1)
q46 = a[46].drop(a[46].columns[[0,1,2]],axis=1)
q47 = a[47].drop(a[47].columns[[0,1,2]],axis=1)
q48 = a[48].drop(a[48].columns[[0,1,2]],axis=1)
q49 = a[49].drop(a[49].columns[[0,1,2]],axis=1)
q50 = a[50].drop(a[50].columns[[0,1,2]],axis=1)
q51 = a[51].drop(a[51].columns[[0,1,2]],axis=1)
q52 = a[52].drop(a[52].columns[[0,1,2]],axis=1)
q53 = a[53].drop(a[53].columns[[0,1,2]],axis=1)
q54 = a[54].drop(a[54].columns[[0,1,2]],axis=1)
q55 = a[55].drop(a[55].columns[[0,1,2]],axis=1)
q56 = a[56].drop(a[56].columns[[0,1,2]],axis=1)
q57 = a[57].drop(a[57].columns[[0,1,2]],axis=1)
q58 = a[58].drop(a[58].columns[[0,1,2]],axis=1)
q59 = a[59].drop(a[59].columns[[0,1,2]],axis=1)
q60 = a[60].drop(a[60].columns[[0,1,2]],axis=1)
q61 = a[61].drop(a[61].columns[[0,1,2]],axis=1)
q62 = a[62].drop(a[62].columns[[0,1,2]],axis=1)
q63 = a[63].drop(a[63].columns[[0,1,2]],axis=1)
q64 = a[64].drop(a[64].columns[[0,1,2]],axis=1)
q65 = a[65].drop(a[65].columns[[0,1,2]],axis=1)
q66 = a[66].drop(a[66].columns[[0,1,2]],axis=1)
q67 = a[67].drop(a[67].columns[[0,1,2]],axis=1)
q68 = a[68].drop(a[68].columns[[0,1,2]],axis=1)
q69 = a[69].drop(a[69].columns[[0,1,2]],axis=1)
q70 = a[70].drop(a[70].columns[[0,1,2]],axis=1)
q71 = a[71].drop(a[71].columns[[0,1,2]],axis=1)
q72= a[72].drop(a[72].columns[[0,1,2]],axis=1)
q73 = a[73].drop(a[73].columns[[0,1,2]],axis=1)
q74 = a[74].drop(a[74].columns[[0,1,2]],axis=1)
q75= a[75].drop(a[75].columns[[0,1,2]],axis=1)
q76 = a[76].drop(a[76].columns[[0,1,2]],axis=1)
q77 = a[77].drop(a[77].columns[[0,1,2]],axis=1)##
q78 = a[78].drop(a[78].columns[[0,1,2]],axis=1)
q79 = a[79].drop(a[79].columns[[0,1,2]],axis=1)
q80 = a[80].drop(a[80].columns[[0,1,2]],axis=1)
q81 = a[81].drop(a[81].columns[[0,1,2]],axis=1)
q82 = a[82].drop(a[82].columns[[0,1,2]],axis=1)
q83 = a[83].drop(a[83].columns[[0,1,2]],axis=1)
q84 = a[84].drop(a[84].columns[[0,1,2]],axis=1)
q85 = a[85].drop(a[85].columns[[0,1,2]],axis=1)
q86 = a[86].drop(a[86].columns[[0,1,2]],axis=1)
q87 = a[87].drop(a[87].columns[[0,1,2]],axis=1)
q88 = a[88].drop(a[88].columns[[0,1,2]],axis=1)
q89 = a[89].drop(a[89].columns[[0,1,2]],axis=1)
q90 = a[90].drop(a[90].columns[[0,1,2]],axis=1)
q91 = a[91].drop(a[91].columns[[0,1,2]],axis=1)
q92 = a[92].drop(a[92].columns[[0,1,2]],axis=1)
q93 = a[93].drop(a[93].columns[[0,1,2]],axis=1)
q94 = a[94].drop(a[94].columns[[0,1,2]],axis=1)
q95 = a[95].drop(a[95].columns[[0,1,2]],axis=1)
q96 = a[96].drop(a[96].columns[[0,1,2]],axis=1)
q97 = a[97].drop(a[97].columns[[0,1,2]],axis=1)
q98 = a[98].drop(a[98].columns[[0,1,2]],axis=1)
q99 = a[99].drop(a[99].columns[[0,1,2]],axis=1)
q100 = a[100].drop(a[100].columns[[0,1,2]],axis=1)
q101= a[101].drop(a[101].columns[[0,1,2]],axis=1)
q102= a[102].drop(a[102].columns[[0,1,2]],axis=1)
q103= a[103].drop(a[103].columns[[0,1,2]],axis=1)
q104= a[104].drop(a[104].columns[[0,1,2]],axis=1)
q105= a[105].drop(a[105].columns[[0,1,2]],axis=1)
q106= a[106].drop(a[106].columns[[0,1,2]],axis=1)
q107= a[107].drop(a[107].columns[[0,1,2]],axis=1)
q108= a[108].drop(a[108].columns[[0,1,2]],axis=1)
q109= a[109].drop(a[109].columns[[0,1,2]],axis=1)
q110= a[110].drop(a[110].columns[[0,1,2]],axis=1)
q111= a[111].drop(a[111].columns[[0,1,2]],axis=1)
q112= a[112].drop(a[112].columns[[0,1,2]],axis=1)
q113= a[113].drop(a[113].columns[[0,1,2]],axis=1)
q114 = a[114].drop(a[114].columns[[0,1,2]],axis=1)
q115 = a[115].drop(a[115].columns[[0,1,2]],axis=1)
q116= a[116].drop(a[116].columns[[0,1,2]],axis=1)
q117= a[117].drop(a[117].columns[[0,1,2]],axis=1)
q118 = a[118].drop(a[118].columns[[0,1,2]],axis=1)
q119= a[119].drop(a[119].columns[[0,1,2]],axis=1)
q120= a[120].drop(a[120].columns[[0,1,2]],axis=1)
q121= a[121].drop(a[121].columns[[0,1,2]],axis=1)
q122= a[122].drop(a[122].columns[[0,1,2]],axis=1)
q123= a[123].drop(a[123].columns[[0,1,2]],axis=1)
q124= a[124].drop(a[124].columns[[0,1,2]],axis=1)
q125= a[125].drop(a[125].columns[[0,1,2]],axis=1)
q126= a[126].drop(a[126].columns[[0,1,2]],axis=1)
q127= a[127].drop(a[127].columns[[0,1,2]],axis=1)
q128= a[128].drop(a[128].columns[[0,1,2]],axis=1)
q129= a[129].drop(a[129].columns[[0,1,2]],axis=1)
q130= a[130].drop(a[130].columns[[0,1,2]],axis=1)
q131 = a[131].drop(a[131].columns[[0,1,2]],axis=1)
q132= a[132].drop(a[132].columns[[0,1,2]],axis=1)
q133= a[133].drop(a[133].columns[[0,1,2]],axis=1)
q134= a[134].drop(a[134].columns[[0,1,2]],axis=1)
q135= a[135].drop(a[135].columns[[0,1,2]],axis=1)
q136 = a[136].drop(a[136].columns[[0,1,2]],axis=1)
q137 = a[137].drop(a[137].columns[[0,1,2]],axis=1)
q138 = a[138].drop(a[138].columns[[0,1,2]],axis=1)
q139 = a[139].drop(a[139].columns[[0,1,2]],axis=1)
q140 = a[140].drop(a[140].columns[[0,1,2]],axis=1)
q141= a[141].drop(a[141].columns[[0,1,2]],axis=1)
q142= a[142].drop(a[142].columns[[0,1,2]],axis=1)
q143= a[143].drop(a[143].columns[[0,1,2]],axis=1)
q144= a[144].drop(a[144].columns[[0,1,2]],axis=1)
q145= a[145].drop(a[145].columns[[0,1,2]],axis=1)
q146= a[146].drop(a[146].columns[[0,1,2]],axis=1)
q147= a[147].drop(a[147].columns[[0,1,2]],axis=1)
q148= a[148].drop(a[148].columns[[0,1,2]],axis=1)
q149= a[149].drop(a[149].columns[[0,1,2]],axis=1)
q150= a[150].drop(a[150].columns[[0,1,2]],axis=1)
q151= a[151].drop(a[151].columns[[0,1,2]],axis=1)
q152= a[152].drop(a[152].columns[[0,1,2]],axis=1)
q153= a[153].drop(a[153].columns[[0,1,2]],axis=1)
q154 = a[154].drop(a[154].columns[[0,1,2]],axis=1)
q155 = a[155].drop(a[155].columns[[0,1,2]],axis=1)
q156= a[156].drop(a[156].columns[[0,1,2]],axis=1)
q157= a[157].drop(a[157].columns[[0,1,2]],axis=1)
q158 = a[158].drop(a[158].columns[[0,1,2]],axis=1)
q159= a[159].drop(a[159].columns[[0,1,2]],axis=1)
q160= a[160].drop(a[160].columns[[0,1,2]],axis=1)
q161= a[161].drop(a[161].columns[[0,1,2]],axis=1)
q162= a[162].drop(a[162].columns[[0,1,2]],axis=1)
q163= a[163].drop(a[163].columns[[0,1,2]],axis=1)
q164= a[164].drop(a[164].columns[[0,1,2]],axis=1)
q165= a[165].drop(a[165].columns[[0,1,2]],axis=1)
q166= a[166].drop(a[166].columns[[0,1,2]],axis=1)
q167= a[167].drop(a[167].columns[[0,1,2]],axis=1)
q168= a[168].drop(a[168].columns[[0,1,2]],axis=1)
q169= a[169].drop(a[169].columns[[0,1,2]],axis=1)
q170= a[170].drop(a[170].columns[[0,1,2]],axis=1)
q171 = a[171].drop(a[171].columns[[0,1,2]],axis=1)

#Only prepared sand data:
r0 = b[0].drop(b[0].columns[[0,1,2]],axis=1)
r1 = b[1].drop(b[1].columns[[0,1,2]],axis=1)
r2 = b[2].drop(b[2].columns[[0,1,2]],axis=1)
r3 = b[3].drop(b[3].columns[[0,1,2]],axis=1)
r4 = b[4].drop(b[4].columns[[0,1,2]],axis=1)
r5 = b[5].drop(b[5].columns[[0,1,2]],axis=1)
r6 = b[6].drop(b[6].columns[[0,1,2]],axis=1)
r7 = b[7].drop(b[7].columns[[0,1,2]],axis=1)
r8 = b[8].drop(b[8].columns[[0,1,2]],axis=1)
r9 = b[9].drop(b[9].columns[[0,1,2]],axis=1)
r10 = b[10].drop(b[10].columns[[0,1,2]],axis=1)
r11 = b[10].drop(b[11].columns[[0,1,2]],axis=1)
r12 = b[12].drop(b[12].columns[[0,1,2]],axis=1)
r13 = b[13].drop(b[13].columns[[0,1,2]],axis=1)
r14 = b[14].drop(b[14].columns[[0,1,2]],axis=1)
r15 = b[15].drop(b[15].columns[[0,1,2]],axis=1)
r16 = b[16].drop(b[16].columns[[0,1,2]],axis=1)
r17 = b[17].drop(b[17].columns[[0,1,2]],axis=1)
r18 = b[18].drop(b[18].columns[[0,1,2]],axis=1)
r19 = b[19].drop(b[19].columns[[0,1,2]],axis=1)
r20 = b[20].drop(b[20].columns[[0,1,2]],axis=1)
r21 = b[21].drop(b[21].columns[[0,1,2]],axis=1)
r22 = b[22].drop(b[22].columns[[0,1,2]],axis=1)
r23 = b[23].drop(b[23].columns[[0,1,2]],axis=1)
r24 = b[24].drop(b[24].columns[[0,1,2]],axis=1)
r25 = b[25].drop(b[25].columns[[0,1,2]],axis=1)
r26 = b[26].drop(b[26].columns[[0,1,2]],axis=1)
r27 = b[27].drop(b[27].columns[[0,1,2]],axis=1)
r28 = b[28].drop(b[28].columns[[0,1,2]],axis=1)
r29 = b[29].drop(b[29].columns[[0,1,2]],axis=1)
r30 = b[30].drop(b[30].columns[[0,1,2]],axis=1)
r31 = b[31].drop(b[31].columns[[0,1,2]],axis=1)
r32 = b[32].drop(b[32].columns[[0,1,2]],axis=1)
r33 = b[33].drop(b[33].columns[[0,1,2]],axis=1)
r34 = b[34].drop(b[34].columns[[0,1,2]],axis=1)
r35 = b[35].drop(b[35].columns[[0,1,2]],axis=1)
r36 = b[36].drop(b[36].columns[[0,1,2]],axis=1)
r37 = b[37].drop(b[37].columns[[0,1,2]],axis=1)
r38 = b[38].drop(b[38].columns[[0,1,2]],axis=1)
r39 = b[39].drop(b[39].columns[[0,1,2]],axis=1)
r40 = b[40].drop(b[40].columns[[0,1,2]],axis=1)
r41 = b[41].drop(b[41].columns[[0,1,2]],axis=1)
r42 = b[42].drop(b[42].columns[[0,1,2]],axis=1)
r43 = b[43].drop(b[43].columns[[0,1,2]],axis=1)
r44 = b[44].drop(b[44].columns[[0,1,2]],axis=1)
r45 = b[45].drop(b[45].columns[[0,1,2]],axis=1)
r46 = b[46].drop(b[46].columns[[0,1,2]],axis=1)
r47 = b[47].drop(b[47].columns[[0,1,2]],axis=1)
r48 = b[48].drop(b[48].columns[[0,1,2]],axis=1)
r49 = b[49].drop(b[49].columns[[0,1,2]],axis=1)
r50 = b[50].drop(b[50].columns[[0,1,2]],axis=1)
r51 = b[51].drop(b[51].columns[[0,1,2]],axis=1)
r52 = b[52].drop(b[52].columns[[0,1,2]],axis=1)
r53 = b[53].drop(b[53].columns[[0,1,2]],axis=1)
r54 = b[54].drop(b[54].columns[[0,1,2]],axis=1)
r55 = b[55].drop(b[55].columns[[0,1,2]],axis=1)
r56 = b[56].drop(b[56].columns[[0,1,2]],axis=1)
r57 = b[57].drop(b[57].columns[[0,1,2]],axis=1)
r58 = b[58].drop(b[58].columns[[0,1,2]],axis=1)
r59 = b[59].drop(b[59].columns[[0,1,2]],axis=1)
r60 = b[60].drop(b[60].columns[[0,1,2]],axis=1)
r61 = b[61].drop(b[61].columns[[0,1,2]],axis=1)
r62 = b[62].drop(b[62].columns[[0,1,2]],axis=1)
r63 = b[63].drop(b[63].columns[[0,1,2]],axis=1)
r64 = b[64].drop(b[64].columns[[0,1,2]],axis=1)
r65 = b[65].drop(b[65].columns[[0,1,2]],axis=1)
r66 = b[66].drop(b[66].columns[[0,1,2]],axis=1)
r67 = b[67].drop(b[67].columns[[0,1,2]],axis=1)
r68 = b[68].drop(b[68].columns[[0,1,2]],axis=1)
r69 = b[69].drop(b[69].columns[[0,1,2]],axis=1)
r70 = b[70].drop(b[70].columns[[0,1,2]],axis=1)
r71 = b[71].drop(b[71].columns[[0,1,2]],axis=1)
r72= b[72].drop(b[72].columns[[0,1,2]],axis=1)
r73 = b[73].drop(b[73].columns[[0,1,2]],axis=1)
r74 = b[74].drop(b[74].columns[[0,1,2]],axis=1)
r75= b[75].drop(b[75].columns[[0,1,2]],axis=1)
r76 = b[76].drop(b[76].columns[[0,1,2]],axis=1)
r77 = b[77].drop(b[77].columns[[0,1,2]],axis=1)##
r78 = b[78].drop(b[78].columns[[0,1,2]],axis=1)
r79 = b[79].drop(b[79].columns[[0,1,2]],axis=1)
r80 = b[80].drop(b[80].columns[[0,1,2]],axis=1)
r81 = b[81].drop(b[81].columns[[0,1,2]],axis=1)
r82 = b[82].drop(b[82].columns[[0,1,2]],axis=1)
r83 = b[83].drop(b[83].columns[[0,1,2]],axis=1)
r84 = b[84].drop(b[84].columns[[0,1,2]],axis=1)
r85 = b[85].drop(b[85].columns[[0,1,2]],axis=1)
r86 = b[86].drop(b[86].columns[[0,1,2]],axis=1)
r87 = b[87].drop(b[87].columns[[0,1,2]],axis=1)
r88 = b[88].drop(b[88].columns[[0,1,2]],axis=1)
r89 = b[89].drop(b[89].columns[[0,1,2]],axis=1)
r90 = b[90].drop(b[90].columns[[0,1,2]],axis=1)
r91 = b[91].drop(b[91].columns[[0,1,2]],axis=1)
r92 = b[92].drop(b[92].columns[[0,1,2]],axis=1)
r93 = b[93].drop(b[93].columns[[0,1,2]],axis=1)
r94 = b[94].drop(b[94].columns[[0,1,2]],axis=1)
r95 = b[95].drop(b[95].columns[[0,1,2]],axis=1)
r96 = b[96].drop(b[96].columns[[0,1,2]],axis=1)
r97 = b[97].drop(b[97].columns[[0,1,2]],axis=1)
r98 = b[98].drop(b[98].columns[[0,1,2]],axis=1)
r99 = b[99].drop(b[99].columns[[0,1,2]],axis=1)
r100 = b[100].drop(b[100].columns[[0,1,2]],axis=1)
r101= b[101].drop(b[101].columns[[0,1,2]],axis=1)
r102= b[102].drop(b[102].columns[[0,1,2]],axis=1)
r103= b[103].drop(b[103].columns[[0,1,2]],axis=1)
r104= b[104].drop(b[104].columns[[0,1,2]],axis=1)
r105= b[105].drop(b[105].columns[[0,1,2]],axis=1)
r106= b[106].drop(b[106].columns[[0,1,2]],axis=1)
r107= b[107].drop(b[107].columns[[0,1,2]],axis=1)
r108= b[108].drop(b[108].columns[[0,1,2]],axis=1)
r109= b[109].drop(b[109].columns[[0,1,2]],axis=1)
r110= b[110].drop(b[110].columns[[0,1,2]],axis=1)
r111= b[111].drop(b[111].columns[[0,1,2]],axis=1)
r112= b[112].drop(b[112].columns[[0,1,2]],axis=1)
r113= b[113].drop(b[113].columns[[0,1,2]],axis=1)
r114 = b[114].drop(b[114].columns[[0,1,2]],axis=1)
r115 = b[115].drop(b[115].columns[[0,1,2]],axis=1)
r116= b[116].drop(b[116].columns[[0,1,2]],axis=1)
r117= b[117].drop(b[117].columns[[0,1,2]],axis=1)
r118 = b[118].drop(b[118].columns[[0,1,2]],axis=1)
r119= b[119].drop(b[119].columns[[0,1,2]],axis=1)
r120= b[120].drop(b[120].columns[[0,1,2]],axis=1)
r121= b[121].drop(b[121].columns[[0,1,2]],axis=1)
r122= b[122].drop(b[122].columns[[0,1,2]],axis=1)
r123= b[123].drop(b[123].columns[[0,1,2]],axis=1)
r124= b[124].drop(b[124].columns[[0,1,2]],axis=1)
r125= b[125].drop(b[125].columns[[0,1,2]],axis=1)
r126= b[126].drop(b[126].columns[[0,1,2]],axis=1)
r127= b[127].drop(b[127].columns[[0,1,2]],axis=1)
r128= b[128].drop(b[128].columns[[0,1,2]],axis=1)
r129= b[129].drop(b[129].columns[[0,1,2]],axis=1)
r130= b[130].drop(b[130].columns[[0,1,2]],axis=1)
r131 = b[131].drop(b[131].columns[[0,1,2]],axis=1)
r132= b[132].drop(b[132].columns[[0,1,2]],axis=1)
r133= b[133].drop(b[133].columns[[0,1,2]],axis=1)
r134= b[134].drop(b[134].columns[[0,1,2]],axis=1)
r135= b[135].drop(b[135].columns[[0,1,2]],axis=1)
r136 = b[136].drop(b[136].columns[[0,1,2]],axis=1)
r137 = b[137].drop(b[137].columns[[0,1,2]],axis=1)
r138 = b[138].drop(b[138].columns[[0,1,2]],axis=1)
r139 = b[139].drop(b[139].columns[[0,1,2]],axis=1)
r140 = b[140].drop(b[140].columns[[0,1,2]],axis=1)
r141= b[141].drop(b[141].columns[[0,1,2]],axis=1)
r142= b[142].drop(b[142].columns[[0,1,2]],axis=1)
r143= b[143].drop(b[143].columns[[0,1,2]],axis=1)
r144= b[144].drop(b[144].columns[[0,1,2]],axis=1)
r145= b[145].drop(b[145].columns[[0,1,2]],axis=1)
r146= b[146].drop(b[146].columns[[0,1,2]],axis=1)
r147= b[147].drop(b[147].columns[[0,1,2]],axis=1)
r148= b[148].drop(b[148].columns[[0,1,2]],axis=1)
r149= b[149].drop(b[149].columns[[0,1,2]],axis=1)
r150= b[150].drop(b[150].columns[[0,1,2]],axis=1)
r151= b[151].drop(b[151].columns[[0,1,2]],axis=1)
r152= b[152].drop(b[152].columns[[0,1,2]],axis=1)
r153= b[153].drop(b[153].columns[[0,1,2]],axis=1)
r154 = b[154].drop(b[154].columns[[0,1,2]],axis=1)
r155 = b[155].drop(b[155].columns[[0,1,2]],axis=1)
r156= b[156].drop(b[156].columns[[0,1,2]],axis=1)
r157= b[157].drop(b[157].columns[[0,1,2]],axis=1)
r158 = b[158].drop(b[158].columns[[0,1,2]],axis=1)
r159= b[159].drop(b[159].columns[[0,1,2]],axis=1)
r160= b[160].drop(b[160].columns[[0,1,2]],axis=1)
r161= b[161].drop(b[161].columns[[0,1,2]],axis=1)
r162= b[162].drop(b[162].columns[[0,1,2]],axis=1)
r163= b[163].drop(b[163].columns[[0,1,2]],axis=1)
r164= b[164].drop(b[164].columns[[0,1,2]],axis=1)
r165= b[165].drop(b[165].columns[[0,1,2]],axis=1)
r166= b[166].drop(b[166].columns[[0,1,2]],axis=1)
r167= b[167].drop(b[167].columns[[0,1,2]],axis=1)
r168= b[168].drop(b[168].columns[[0,1,2]],axis=1)
r169= b[169].drop(b[169].columns[[0,1,2]],axis=1)
r170= b[170].drop(b[170].columns[[0,1,2]],axis=1)
r171 = b[171].drop(b[171].columns[[0,1,2]],axis=1)


#psepased sand and bentonite datafsames:
s0 = c[0].drop(c[0].columns[[0,1,2]],axis=1)
s1 = c[1].drop(c[1].columns[[0,1,2]],axis=1)
s2 = c[2].drop(c[2].columns[[0,1,2]],axis=1)
s3 = c[3].drop(c[3].columns[[0,1,2]],axis=1)
s4 = c[4].drop(c[4].columns[[0,1,2]],axis=1)
s5 = c[5].drop(c[5].columns[[0,1,2]],axis=1)
s6 = c[6].drop(c[6].columns[[0,1,2]],axis=1)
s7 = c[7].drop(c[7].columns[[0,1,2]],axis=1)
s8 = c[8].drop(c[8].columns[[0,1,2]],axis=1)
s9 = c[9].drop(c[9].columns[[0,1,2]],axis=1)
s10 = c[10].drop(c[10].columns[[0,1,2]],axis=1)
s11 = c[10].drop(c[11].columns[[0,1,2]],axis=1)
s12 = c[12].drop(c[12].columns[[0,1,2]],axis=1)
s13 = c[13].drop(c[13].columns[[0,1,2]],axis=1)
s14 = c[14].drop(c[14].columns[[0,1,2]],axis=1)
s15 = c[15].drop(c[15].columns[[0,1,2]],axis=1)
s16 = c[16].drop(c[16].columns[[0,1,2]],axis=1)
s17 = c[17].drop(c[17].columns[[0,1,2]],axis=1)
s18 = c[18].drop(c[18].columns[[0,1,2]],axis=1)
s19 = c[19].drop(c[19].columns[[0,1,2]],axis=1)
s20 = c[20].drop(c[20].columns[[0,1,2]],axis=1)
s21 = c[21].drop(c[21].columns[[0,1,2]],axis=1)
s22 = c[22].drop(c[22].columns[[0,1,2]],axis=1)
s23 = c[23].drop(c[23].columns[[0,1,2]],axis=1)
s24 = c[24].drop(c[24].columns[[0,1,2]],axis=1)
s25 = c[25].drop(c[25].columns[[0,1,2]],axis=1)
s26 = c[26].drop(c[26].columns[[0,1,2]],axis=1)
s27 = c[27].drop(c[27].columns[[0,1,2]],axis=1)
s28 = c[28].drop(c[28].columns[[0,1,2]],axis=1)
s29 = c[29].drop(c[29].columns[[0,1,2]],axis=1)
s30 = c[30].drop(c[30].columns[[0,1,2]],axis=1)
s31 = c[31].drop(c[31].columns[[0,1,2]],axis=1)
s32 = c[32].drop(c[32].columns[[0,1,2]],axis=1)
s33 = c[33].drop(c[33].columns[[0,1,2]],axis=1)
s34 = c[34].drop(c[34].columns[[0,1,2]],axis=1)
s35 = c[35].drop(c[35].columns[[0,1,2]],axis=1)
s36 = c[36].drop(c[36].columns[[0,1,2]],axis=1)
s37 = c[37].drop(c[37].columns[[0,1,2]],axis=1)
s38 = c[38].drop(c[38].columns[[0,1,2]],axis=1)
s39 = c[39].drop(c[39].columns[[0,1,2]],axis=1)
s40 = c[40].drop(c[40].columns[[0,1,2]],axis=1)
s41 = c[41].drop(c[41].columns[[0,1,2]],axis=1)
s42 = c[42].drop(c[42].columns[[0,1,2]],axis=1)
s43 = c[43].drop(c[43].columns[[0,1,2]],axis=1)
s44 = c[44].drop(c[44].columns[[0,1,2]],axis=1)
s45 = c[45].drop(c[45].columns[[0,1,2]],axis=1)
s46 = c[46].drop(c[46].columns[[0,1,2]],axis=1)
s47 = c[47].drop(c[47].columns[[0,1,2]],axis=1)
s48 = c[48].drop(c[48].columns[[0,1,2]],axis=1)
s49 = c[49].drop(c[49].columns[[0,1,2]],axis=1)
s50 = c[50].drop(c[50].columns[[0,1,2]],axis=1)
s51 = c[51].drop(c[51].columns[[0,1,2]],axis=1)
s52 = c[52].drop(c[52].columns[[0,1,2]],axis=1)
s53 = c[53].drop(c[53].columns[[0,1,2]],axis=1)
s54 = c[54].drop(c[54].columns[[0,1,2]],axis=1)
s55 = c[55].drop(c[55].columns[[0,1,2]],axis=1)
s56 = c[56].drop(c[56].columns[[0,1,2]],axis=1)
s57 = c[57].drop(c[57].columns[[0,1,2]],axis=1)
s58 = c[58].drop(c[58].columns[[0,1,2]],axis=1)
s59 = c[59].drop(c[59].columns[[0,1,2]],axis=1)
s60 = c[60].drop(c[60].columns[[0,1,2]],axis=1)
s61 = c[61].drop(c[61].columns[[0,1,2]],axis=1)
s62 = c[62].drop(c[62].columns[[0,1,2]],axis=1)
s63 = c[63].drop(c[63].columns[[0,1,2]],axis=1)
s64 = c[64].drop(c[64].columns[[0,1,2]],axis=1)
s65 = c[65].drop(c[65].columns[[0,1,2]],axis=1)
s66 = c[66].drop(c[66].columns[[0,1,2]],axis=1)
s67 = c[67].drop(c[67].columns[[0,1,2]],axis=1)
s68 = c[68].drop(c[68].columns[[0,1,2]],axis=1)
s69 = c[69].drop(c[69].columns[[0,1,2]],axis=1)
s70 = c[70].drop(c[70].columns[[0,1,2]],axis=1)
s71 = c[71].drop(c[71].columns[[0,1,2]],axis=1)
s72= c[72].drop(c[72].columns[[0,1,2]],axis=1)
s73 = c[73].drop(c[73].columns[[0,1,2]],axis=1)
s74 = c[74].drop(c[74].columns[[0,1,2]],axis=1)
s75= c[75].drop(c[75].columns[[0,1,2]],axis=1)
s76 = c[76].drop(c[76].columns[[0,1,2]],axis=1)
s77 = c[77].drop(c[77].columns[[0,1,2]],axis=1)##
s78 = c[78].drop(c[78].columns[[0,1,2]],axis=1)
s79 = c[79].drop(c[79].columns[[0,1,2]],axis=1)
s80 = c[80].drop(c[80].columns[[0,1,2]],axis=1)
s81 = c[81].drop(c[81].columns[[0,1,2]],axis=1)
s82 = c[82].drop(c[82].columns[[0,1,2]],axis=1)
s83 = c[83].drop(c[83].columns[[0,1,2]],axis=1)
s84 = c[84].drop(c[84].columns[[0,1,2]],axis=1)
s85 = c[85].drop(c[85].columns[[0,1,2]],axis=1)
s86 = c[86].drop(c[86].columns[[0,1,2]],axis=1)
s87 = c[87].drop(c[87].columns[[0,1,2]],axis=1)
s88 = c[88].drop(c[88].columns[[0,1,2]],axis=1)
s89 = c[89].drop(c[89].columns[[0,1,2]],axis=1)
s90 = c[90].drop(c[90].columns[[0,1,2]],axis=1)
s91 = c[91].drop(c[91].columns[[0,1,2]],axis=1)
s92 = c[92].drop(c[92].columns[[0,1,2]],axis=1)
s93 = c[93].drop(c[93].columns[[0,1,2]],axis=1)
s94 = c[94].drop(c[94].columns[[0,1,2]],axis=1)
s95 = c[95].drop(c[95].columns[[0,1,2]],axis=1)
s96 = c[96].drop(c[96].columns[[0,1,2]],axis=1)
s97 = c[97].drop(c[97].columns[[0,1,2]],axis=1)
s98 = c[98].drop(c[98].columns[[0,1,2]],axis=1)
s99 = c[99].drop(c[99].columns[[0,1,2]],axis=1)
s100 = c[100].drop(c[100].columns[[0,1,2]],axis=1)
s101= c[101].drop(c[101].columns[[0,1,2]],axis=1)
s102= c[102].drop(c[102].columns[[0,1,2]],axis=1)
s103= c[103].drop(c[103].columns[[0,1,2]],axis=1)
s104= c[104].drop(c[104].columns[[0,1,2]],axis=1)
s105= c[105].drop(c[105].columns[[0,1,2]],axis=1)
s106= c[106].drop(c[106].columns[[0,1,2]],axis=1)
s107= c[107].drop(c[107].columns[[0,1,2]],axis=1)
s108= c[108].drop(c[108].columns[[0,1,2]],axis=1)
s109= c[109].drop(c[109].columns[[0,1,2]],axis=1)
s110= c[110].drop(c[110].columns[[0,1,2]],axis=1)
s111= c[111].drop(c[111].columns[[0,1,2]],axis=1)
s112= c[112].drop(c[112].columns[[0,1,2]],axis=1)
s113= c[113].drop(c[113].columns[[0,1,2]],axis=1)
s114 = c[114].drop(c[114].columns[[0,1,2]],axis=1)
s115 = c[115].drop(c[115].columns[[0,1,2]],axis=1)
s116= c[116].drop(c[116].columns[[0,1,2]],axis=1)
s117= c[117].drop(c[117].columns[[0,1,2]],axis=1)
s118 = c[118].drop(c[118].columns[[0,1,2]],axis=1)
s119= c[119].drop(c[119].columns[[0,1,2]],axis=1)
s120= c[120].drop(c[120].columns[[0,1,2]],axis=1)
s121= c[121].drop(c[121].columns[[0,1,2]],axis=1)
s122= c[122].drop(c[122].columns[[0,1,2]],axis=1)
s123= c[123].drop(c[123].columns[[0,1,2]],axis=1)
s124= c[124].drop(c[124].columns[[0,1,2]],axis=1)
s125= c[125].drop(c[125].columns[[0,1,2]],axis=1)
s126= c[126].drop(c[126].columns[[0,1,2]],axis=1)
s127= c[127].drop(c[127].columns[[0,1,2]],axis=1)
s128= c[128].drop(c[128].columns[[0,1,2]],axis=1)
s129= c[129].drop(c[129].columns[[0,1,2]],axis=1)
s130= c[130].drop(c[130].columns[[0,1,2]],axis=1)
s131 = c[131].drop(c[131].columns[[0,1,2]],axis=1)
s132= c[132].drop(c[132].columns[[0,1,2]],axis=1)
s133= c[133].drop(c[133].columns[[0,1,2]],axis=1)
s134= c[134].drop(c[134].columns[[0,1,2]],axis=1)
s135= c[135].drop(c[135].columns[[0,1,2]],axis=1)
s136 = c[136].drop(c[136].columns[[0,1,2]],axis=1)
s137 = c[137].drop(c[137].columns[[0,1,2]],axis=1)
s138 = c[138].drop(c[138].columns[[0,1,2]],axis=1)
s139 = c[139].drop(c[139].columns[[0,1,2]],axis=1)
s140 = c[140].drop(c[140].columns[[0,1,2]],axis=1)
s141= c[141].drop(c[141].columns[[0,1,2]],axis=1)
s142= c[142].drop(c[142].columns[[0,1,2]],axis=1)
s143= c[143].drop(c[143].columns[[0,1,2]],axis=1)
s144= c[144].drop(c[144].columns[[0,1,2]],axis=1)
s145= c[145].drop(c[145].columns[[0,1,2]],axis=1)
s146= c[146].drop(c[146].columns[[0,1,2]],axis=1)
s147= c[147].drop(c[147].columns[[0,1,2]],axis=1)
s148= c[148].drop(c[148].columns[[0,1,2]],axis=1)
s149= c[149].drop(c[149].columns[[0,1,2]],axis=1)
s150= c[150].drop(c[150].columns[[0,1,2]],axis=1)
s151= c[151].drop(c[151].columns[[0,1,2]],axis=1)
s152= c[152].drop(c[152].columns[[0,1,2]],axis=1)
s153= c[153].drop(c[153].columns[[0,1,2]],axis=1)
s154 = c[154].drop(c[154].columns[[0,1,2]],axis=1)
s155 = c[155].drop(c[155].columns[[0,1,2]],axis=1)
s156= c[156].drop(c[156].columns[[0,1,2]],axis=1)
s157= c[157].drop(c[157].columns[[0,1,2]],axis=1)
s158 = c[158].drop(c[158].columns[[0,1,2]],axis=1)
s159= c[159].drop(c[159].columns[[0,1,2]],axis=1)
s160= c[160].drop(c[160].columns[[0,1,2]],axis=1)
s161= c[161].drop(c[161].columns[[0,1,2]],axis=1)
s162= c[162].drop(c[162].columns[[0,1,2]],axis=1)
s163= c[163].drop(c[163].columns[[0,1,2]],axis=1)
s164= c[164].drop(c[164].columns[[0,1,2]],axis=1)
s165= c[165].drop(c[165].columns[[0,1,2]],axis=1)
s166= c[166].drop(c[166].columns[[0,1,2]],axis=1)
s167= c[167].drop(c[167].columns[[0,1,2]],axis=1)
s168= c[168].drop(c[168].columns[[0,1,2]],axis=1)
s169= c[169].drop(c[169].columns[[0,1,2]],axis=1)
s170= c[170].drop(c[170].columns[[0,1,2]],axis=1)
s171 = c[171].drop(c[171].columns[[0,1,2]],axis=1)


bentonite = [q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29,
             q30,q31,q32,q33,q34,q35,q36,q37,q38,q39,q40,q41,q42,q43,q44,q45,q46,q47,q48,q49,q50,q51,q52,q53,q54,q55,q56,q57,
            q58,q59,q60,q61,q62,q63,q64,q65,q66,q67,q68,q69,q70,q71,q72,q73,q74,q75,q76,q77,q78,q79,q80,q81,q82,q83,q84,q85,
            q86,q87,q88,q89,q90,q91,q92,q93,q94,q95,q96,q97,q98,q99,q100,q101,q102,q103,q104,q105,q106,q107,q108,q109,q110,
            q111,q112,q113,q114,q115,q116,q117,q118,q119,q120,q121,q122,q123,q124,q125,q126,q127,q128,q129,q130,q131,q132,q133,
            q134,q135,q136,q137,q138,q139,q140,q141,q142,q143,q144,q145,q146,q147,q148,q149,q150,q151,q152,q153,q154,q155,q156,
            q157,q158,q159,q160,q161,q162,q163,q164,q165,q166,q167,q168,q169,q170,q171]


sand = [r0,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13,r14,r15,r16,r17,r18,r19,r20,r21,r22,r23,r24,r25,r26,r27,r28,r29,
             r30,r31,r32,r33,r34,r35,r36,r37,r38,r39,r40,r41,r42,r43,r44,r45,r46,r47,r48,r49,r50,r51,r52,r53,r54,r55,r56,r57,
            r58,r59,r60,r61,r62,r63,r64,r65,r66,r67,r68,r69,r70,r71,r72,r73,r74,r75,r76,r77,r78,r79,r80,r81,r82,r83,r84,r85,
            r86,r87,r88,r89,r90,r91,r92,r93,r94,r95,r96,r97,r98,r99,r100,r101,r102,r103,r104,r105,r106,r107,r108,r109,r110,
            r111,r112,r113,r114,r115,r116,r117,r118,r119,r120,r121,r122,r123,r124,r125,r126,r127,r128,r129,r130,r131,r132,r133,
            r134,r135,r136,r137,r138,r139,r140,r141,r142,r143,r144,r145,r146,r147,r148,r149,r150,r151,r152,r153,r154,r155,r156,
            r157,r158,r159,r160,r161,r162,r163,r164,r165,r166,r167,r168,r169,r170,r171]


fss = [s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,
             s30,s31,s32,s33,s34,s35,s36,s37,s38,s39,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49,s50,s51,s52,s53,s54,s55,s56,s57,
            s58,s59,s60,s61,s62,s63,s64,s65,s66,s67,s68,s69,s70,s71,s72,s73,s74,s75,s76,s77,s78,s79,s80,s81,s82,s83,s84,s85,
            s86,s87,s88,s89,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99,s100,s101,s102,s103,s104,s105,s106,s107,s108,s109,s110,
            s111,s112,s113,s114,s115,s116,s117,s118,s119,s120,s121,s122,s123,s124,s125,s126,s127,s128,s129,s130,s131,s132,s133,
            s134,s135,s136,s137,s138,s139,s140,s141,s142,s143,s144,s145,s146,s147,s148,s149,s150,s151,s152,s153,s154,s155,s156,
            s157,s158,s159,s160,s161,s162,s163,s164,s165,s166,s167,s168,s169,s170,s171]

d2= d1[['gcs_', 'compactability_', 'active_clay_', 'wet_tensile_strength_', 'moisture_', 'permeability']]
d3 = d1[['loi_', 'inert_fines_', 'volatile_matter_']]
d4 = d1[['gfnafs']]

df_names = []

comp1={}
for j in bentonite:
    if len(j)<5:
        print(j['component_id'],':','Insufficient data to build a model')
        continue
    else:
        pass
        
        for i in j:
            if i!='component_id':
                X=j.drop([i,'component_id'],axis=1) 
                y = j[i]
                if i == 'loi_':
                    break 

                sc=StandardScaler()
                X=sc.fit_transform(X)
                X = pd.DataFrame(X)

                from sklearn.model_selection import KFold 
                from sklearn import metrics
                n_split = 5 
                kf=KFold(n_splits=n_split,shuffle=True,random_state=0) 

                for model,name in zip([boost_rf],['boosted_random_forest']):
                    rmse_test=[]
                    r2_test=[]
                    adj_r2_test=[]
                    rmse_train=[]
                    r2_train=[]
                    adj_r2_train=[]
                    
                    for train,test in kf.split(X,y):
                        X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                        y_train,y_test=y.iloc[train],y.iloc[test]
                        model.fit(X_train,y_train) #fitting the model
                        y_predict_test=model.predict(X_test) #predicting the test result
                        mse_test=round(metrics.mean_squared_error(y_test,y_predict_test),3) #calculating mse
                        rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                        rSquare_test = round(r2_score((y_test),(y_predict_test)),3) #calculating rsquared for test
                        adj_rsquare_test= round(1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for test
                        r2_test.append(rSquare_test) #appending the result to the empty list
                        adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list

                        y_predict_train=model.predict(X_train) #predicting the result for train
                        mse_train=round(metrics.mean_squared_error(y_train,y_predict_train),3) #calculating mse for train
                        rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                        rSquare_train = round(r2_score((y_train),(y_predict_train)),3) #calculating rsquared for train
                        adj_rsquare_train= round(1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for train
                        r2_train.append(rSquare_train) #appending the result to the empty list
                        adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                    exec(f"df_{name}_{i} = pd.DataFrame()")
                    exec(f"df_{name}_{i}['Name'] =[name]*n_split")
                    exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
                    exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
                    exec(f"df_{name}_{i}['r2_train'] =r2_train")
                    exec(f"df_{name}_{i}['r2_test'] =r2_test")
                    exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
                    exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
                    exec(f"df_{name}_{i}['output'] = [i]*n_split")
                    df_names.append(f"df_{name}_{i}")
            
            
    collection = pd.DataFrame() #collecting the results in dataframe

    for nn in df_names:
        exec(f"collection=pd.concat([collection,{nn}])")

        collection['new_name'] = collection['output']


    df4 = collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

    abc = list(d2.columns)

    new = pd.DataFrame(columns=df4.columns)
    for names in range(len(abc)):
        val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
        df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
        new=pd.concat([new,df5])           
    
    comp=j.component_id.unique()[0]
    comp1[comp]=new
    print('\n')
    print('\033[1m'+ comp+':')
    print('\n')
    new[['r2_train','r2_test','adj_r2_train','adj_r2_test']] = round(new[['r2_train','r2_test','adj_r2_train','adj_r2_test']]*100,2)
    new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
    print(new)
    print('\n')
    
df_names = []

comp1={}
for j in sand:
    if len(j)<5:
        print(j['component_id'],':','Insufficient data to build a model')
        continue
    else:
        pass
        
        for i in j:
            if i!='component_id':
                X=j.drop([i,'component_id'],axis=1) 
                y = j[i]
                if i == 'gcs_':
                    break 

                sc=StandardScaler()
                X=sc.fit_transform(X)
                X = pd.DataFrame(X)

                from sklearn.model_selection import KFold 
                from sklearn import metrics
                n_split = 5 
                kf=KFold(n_splits=n_split,shuffle=True,random_state=0) 

                for model,name in zip([boost_rf],['boosted_random_forest']):
                    rmse_test=[]
                    r2_test=[]
                    adj_r2_test=[]
                    rmse_train=[]
                    r2_train=[]
                    adj_r2_train=[]
                    
                    for train,test in kf.split(X,y):
                        X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                        y_train,y_test=y.iloc[train],y.iloc[test]
                        model.fit(X_train,y_train) #fitting the model
                        y_predict_test=model.predict(X_test) #predicting the test result
                        mse_test=round(metrics.mean_squared_error(y_test,y_predict_test),3) #calculating mse
                        rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                        rSquare_test = round(r2_score((y_test),(y_predict_test)),3) #calculating rsquared for test
                        adj_rsquare_test= round(1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for test
                        r2_test.append(rSquare_test) #appending the result to the empty list
                        adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list

                        y_predict_train=model.predict(X_train) #predicting the result for train
                        mse_train=round(metrics.mean_squared_error(y_train,y_predict_train),3) #calculating mse for train
                        rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                        rSquare_train = round(r2_score((y_train),(y_predict_train)),3) #calculating rsquared for train
                        adj_rsquare_train= round(1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for train
                        r2_train.append(rSquare_train) #appending the result to the empty list
                        adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                    exec(f"df_{name}_{i} = pd.DataFrame()")
                    exec(f"df_{name}_{i}['Name'] =[name]*n_split")
                    exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
                    exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
                    exec(f"df_{name}_{i}['r2_train'] =r2_train")
                    exec(f"df_{name}_{i}['r2_test'] =r2_test")
                    exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
                    exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
                    exec(f"df_{name}_{i}['output'] = [i]*n_split")
                    df_names.append(f"df_{name}_{i}")
            
            
    collection = pd.DataFrame() #collecting the results in dataframe

    for nn in df_names:
        exec(f"collection=pd.concat([collection,{nn}])")

        collection['new_name'] = collection['output']


    df4 = collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

    abc = list(d3.columns)

    new = pd.DataFrame(columns=df4.columns)
    for names in range(len(abc)):
        val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
        df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
        new=pd.concat([new,df5])           
    
    comp=j.component_id.unique()[0]
    comp1[comp]=new
    print('\n')
    print('\033[1m'+ comp+':')
    print('\n')
    new[['r2_train','r2_test','adj_r2_train','adj_r2_test']] = round(new[['r2_train','r2_test','adj_r2_train','adj_r2_test']]*100,2)
    new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
    print(new)
    print('\n')
    
df_names = []

comp1={}
for j in fss:
    if len(j)<5:
        print(j['component_id'],':','Insufficient data to build a model')
        continue
    else:
        pass
        
        for i in j:
            if i!='component_id':
                X=j.drop([i,'component_id'],axis=1) 
                y = j[i]
                if i == 'gcs_':
                    break 

                sc=StandardScaler()
                X=sc.fit_transform(X)
                X = pd.DataFrame(X)

                from sklearn.model_selection import KFold 
                from sklearn import metrics
                n_split = 5 
                kf=KFold(n_splits=n_split,shuffle=True,random_state=0) 

                for model,name in zip([boost_rf],['boosted_random_forest']):
                    rmse_test=[]
                    r2_test=[]
                    adj_r2_test=[]
                    rmse_train=[]
                    r2_train=[]
                    adj_r2_train=[]
                    
                    for train,test in kf.split(X,y):
                        X_train,X_test=X.iloc[train,:],X.iloc[test,:]
                        y_train,y_test=y.iloc[train],y.iloc[test]
                        model.fit(X_train,y_train) #fitting the model
                        y_predict_test=model.predict(X_test) #predicting the test result
                        mse_test=round(metrics.mean_squared_error(y_test,y_predict_test),3) #calculating mse
                        rmse_test.append(np.sqrt(mse_test)) # calculating rmse for test
                        rSquare_test = round(r2_score((y_test),(y_predict_test)),3) #calculating rsquared for test
                        adj_rsquare_test= round(1-(1-rSquare_test)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for test
                        r2_test.append(rSquare_test) #appending the result to the empty list
                        adj_r2_test.append(adj_rsquare_test) #appending the result to the empty list

                        y_predict_train=model.predict(X_train) #predicting the result for train
                        mse_train=round(metrics.mean_squared_error(y_train,y_predict_train),3) #calculating mse for train
                        rmse_train.append(np.sqrt(mse_train)) # calculating rmse for train
                        rSquare_train = round(r2_score((y_train),(y_predict_train)),3) #calculating rsquared for train
                        adj_rsquare_train= round(1-(1-rSquare_train)*(len(y)-1)/(len(y)-X.shape[1]-1),3) #calculating adjusted r2 for train
                        r2_train.append(rSquare_train) #appending the result to the empty list
                        adj_r2_train.append(adj_rsquare_train) #appending the result to the empty list
                
                
                    exec(f"df_{name}_{i} = pd.DataFrame()")
                    exec(f"df_{name}_{i}['Name'] =[name]*n_split")
                    exec(f"df_{name}_{i}['rmse_train'] =rmse_train")
                    exec(f"df_{name}_{i}['rmse_test'] =rmse_test")
                    exec(f"df_{name}_{i}['r2_train'] =r2_train")
                    exec(f"df_{name}_{i}['r2_test'] =r2_test")
                    exec(f"df_{name}_{i}['adj_r2_train'] =adj_r2_train")
                    exec(f"df_{name}_{i}['adj_r2_test'] =adj_r2_test")
                    exec(f"df_{name}_{i}['output'] = [i]*n_split")
                    df_names.append(f"df_{name}_{i}")
            
            
    collection = pd.DataFrame() #collecting the results in dataframe

    for nn in df_names:
        exec(f"collection=pd.concat([collection,{nn}])")

        collection['new_name'] = collection['output']


    df4 = collection.groupby(['new_name']).mean()

#finding the mean of the results to find the overall best model
#df4['Mean'] = (df4['rmse_train'] + df4['rmse_test'] + (1-df4['r_squared_train']) + (1-df4['r_squared_test']))/4
#df4.sort_values(by='Mean',inplace=True)

    abc = list(d4.columns)

    new = pd.DataFrame(columns=df4.columns)
    for names in range(len(abc)):
        val = df4[df4.index.isin([i for i in df4.index if abc[names] in i])].iloc[0,:]
        df5=pd.DataFrame(val.values.reshape(1,-1),columns=new.columns,index=[val.name])
        new=pd.concat([new,df5])           
    
    comp=j.component_id.unique()[0]
    comp1[comp]=new
    print('\n')
    print('\033[1m'+ comp+':')
    print('\n')
    new[['r2_train','r2_test','adj_r2_train','adj_r2_test']] = round(new[['r2_train','r2_test','adj_r2_train','adj_r2_test']]*100,2)
    new[['rmse_train', 'rmse_test']] = round(new[['rmse_train', 'rmse_test']],2)
    print(new)
    print('\n')


All
                       rmse_train  rmse_test  r_squared_train  r_squared_test  \
gcs_                         0.02       0.04            93.29           76.73   
compactability_              0.40       0.68            88.92           67.94   
active_clay_                 0.00       0.01           100.00           99.79   
wet_tensile_strength_        0.10       0.45            94.61            1.55   
moisture_                    0.02       0.04            89.25           65.34   
permeability                 1.36       2.73            91.53           65.95   
loi_                         0.00       0.03            99.99           97.45   
inert_fines_                 0.00       0.01           100.00           99.90   
volatile_matter_             0.00       0.02            99.98           95.68   
gfnafs                       0.02       0.27            99.97           94.04   

                       adj_r2_train  adj_r2_test  
gcs_                          93.27        76.65  
co

C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Group High:
                       rmse_train  rmse_test  r_squared_train  r_squared_test  \
gcs_                         0.02       0.06            97.20           74.50   
compactability_              0.18       0.81            98.28           63.77   
active_clay_                 0.01       0.06            99.83           95.38   
wet_tensile_strength_        0.14       0.77            96.73          -15.16   
moisture_                    0.01       0.06            97.99           67.66   
permeability                 0.92       3.98            96.35           30.02   
loi_                         0.02       0.10            99.30           78.36   
inert_fines_                 0.01       0.06            99.92           97.16   
volatile_matter_             0.01       0.06            99.05           69.40   
gfnafs                       0.11       0.50            98.82           74.02   

                       adj_r2_train  adj_r2_test  
gcs_                          97.08        73

C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Group Medium:
                       rmse_train  rmse_test  r_squared_train  r_squared_test  \
gcs_                         0.02       0.04            95.53           83.50   
compactability_              0.24       0.63            92.88           49.61   
active_clay_                 0.01       0.05            99.96           96.81   
wet_tensile_strength_        0.05       0.42            98.56          -20.35   
moisture_                    0.03       0.05            83.59           45.15   
permeability                 1.43       2.92            90.85           61.92   
loi_                         0.01       0.07            99.79           89.06   
inert_fines_                 0.00       0.05            99.98           98.32   
volatile_matter_             0.01       0.04            99.68           84.35   
gfnafs                       0.06       0.50            99.75           83.24   

                       adj_r2_train  adj_r2_test  
gcs_                          95.46        

...



GAC013:


                       adj_r2_test  adj_r2_train  r2_test  r2_train  \
gcs_                         180.4        110.12    -60.8     79.74   
compactability_              150.0        134.56      0.0     30.86   
active_clay_                 332.2        133.66   -364.5     32.68   
wet_tensile_strength_        100.0        129.08    100.0     41.80   
moisture_                    150.0        132.32      0.0     35.34   
permeability                 218.4        133.30   -136.7     33.40   

                       rmse_test  rmse_train  
gcs_                        0.04        0.01  
compactability_             0.50        0.38  
active_clay_                0.05        0.04  
wet_tensile_strength_       0.18        0.17  
moisture_                   0.05        0.03  
permeability                3.67        2.19  


1025    RAM003
Name: component_id, dtype: object : Insufficient data to build a model


SEA003:


                       adj_r2_test  adj_r2_train  r2_test  r2

C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\r



RAE029:


                       adj_r2_test  adj_r2_train   r2_test  r2_train  \
gcs_                      15419.50        136.94 -22879.20     44.60   
compactability_             357.50        147.06   -286.20     29.42   
active_clay_                798.95        127.66   -948.40     58.50   
wet_tensile_strength_       193.40        134.04    -40.10     48.94   
moisture_                   249.25        124.26   -123.85     63.62   
permeability               2206.30        116.58  -3059.50     75.16   

                       rmse_test  rmse_train  
gcs_                        0.04        0.03  
compactability_             1.23        1.05  
active_clay_                0.18        0.17  
wet_tensile_strength_       0.47        0.33  
moisture_                   0.06        0.02  
permeability                2.00        1.51  




SGC004:


                       adj_r2_test  adj_r2_train  r2_test  r2_train  \
gcs_                       1231.78        128.34   -74.12     95.64  

C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\91979\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\r



RAB010:


                       adj_r2_test  adj_r2_train  r2_test  r2_train  \
gcs_                         572.0        131.00   -844.0     38.00   
compactability_              292.6        126.98   -285.3     46.06   
active_clay_                 174.4        121.32    -48.7     57.38   
wet_tensile_strength_        100.0        157.66    100.0    -15.28   
moisture_                    150.0        140.32      0.0     19.38   
permeability                 345.4        124.14   -390.8     51.72   

                       rmse_test  rmse_train  
gcs_                        0.10        0.05  
compactability_             0.89        0.65  
active_clay_                0.08        0.05  
wet_tensile_strength_       0.12        0.10  
moisture_                   0.08        0.05  
permeability                1.67        0.95  




MAR007:


                       adj_r2_test  adj_r2_train  r2_test  r2_train  \
gcs_                       -148.82         88.86   -20.12     94.62   
compa